In [20]:
import os
os.environ['PROJ_LIB'] = r'C:\Users\HP\anaconda3\pkgs\proj-6.2.1-h9f7ef89_0\Library\share\proj'
# # os.environ['PROJ_LIB'] = r'C:\Users\username\AppData\Local\Continuum\anaconda2\pkgs\proj4-4.9.2-vc10_0\Library\share'
# pip install matplotlib==3.1.3

In [25]:
import matplotlib.pyplot as plt
print('version statmodel',plt.__version__)

AttributeError: module 'matplotlib.pyplot' has no attribute '__version__'

In [21]:
import numpy as np
import pandas as pd
# import matplotlib.pyplot as plt
# from matplotlib import colors
# import math
# import csv
# %matplotlib inline
from mpl_toolkits.basemap import Basemap
from mpl_toolkits.axes_grid1 import make_axes_locatable

ImportError: cannot import name 'dedent' from 'matplotlib.cbook' (C:\Users\HP\anaconda3\lib\site-packages\matplotlib\cbook\__init__.py)

In [12]:
#Types de projections existant
# np.zeros((300, 250, 3), dtype = int)
# #Les différentes projections
# plt.figure(figsize=(8, 8))
# # m = Basemap(projection='ortho', resolution=None, lat_0=50, lon_0=-100)
# # m = Basemap(projection='moll', resolution='c', lon_0=0)
# # m = Basemap(projection='robin', resolution='c', lon_0=0)
# m = Basemap(projection='cyl',llcrnrlat=-90,urcrnrlat=90,\
#             llcrnrlon=-180,urcrnrlon=180,resolution='c')
# m.bluemarble(scale=0.5);

In [13]:
#on télécharge le dataset comprenant les coordonnées GPS de chaque capitale
url = 'INPUTS_FORMATION/LONG_LAT_numeric_plus.csv'
# data = pd.read_csv(url, sep=';',encoding='utf-8')
data = pd.read_csv(url, sep=';',encoding = 'ISO-8859-1') #prendre en cpte les accents
df = data.copy()
df['Zone'] = df['Zone'].str.title()
df['Capitale'] = df['Capitale'].str.title()
# df['Zone'].unique()

In [14]:
#on télécharge le dataset final issu du Projet
url = 'INPUTS_FORMATION/post_treatment.csv'
data = pd.read_csv(url, sep=',',encoding='utf-8')
# data = pd.read_csv(url, sep=',',encoding = 'ISO-8859-1') #prendre en cpte les accents
table = data.copy()
# table['Zone'].unique()

In [15]:
#on renomme quelques Zones pour être conforme au dataset 'GPS'
table['Zone'] = table['Zone'].replace("Bolivie (État plurinational de)" ,'Bolivie')
table['Zone'] = table['Zone'].replace("Venezuela (République bolivarienne du)" ,'Venezuela')
table['Zone'] = table['Zone'].replace("Iran (République islamique d')",'Iran')
table['Zone'] = table['Zone'].replace("Tchéquie (la)",'Tchecoslovaquie')
table['Zone'] = table['Zone'].replace("Chine - RAS de Hong-Kong",'Hong Kong')
table['Zone'] = table['Zone'].replace("Chine - RAS de Macao",'Macao')
table['Zone'] = table['Zone'].replace("Chine, Taiwan Province de",'Taiwan')
table['Zone'] = table['Zone'].replace("République populaire démocratique de Corée",'Corée du Nord')
table['Zone'] = table['Zone'].replace("République de Corée",'Corée du Sud')
table['Zone'] = table['Zone'].str.title()
# vecteur_zone_renamed=table['Zone'].unique()
# table['Zone'].to_csv("OUTPUTS_FORMATION/P3_zone_tabl_renommee.csv", index = False,encoding = "ISO-8859-1")

In [16]:
############################ PREPARATION DU DF POUR LA SS-ALIMENTATION ############################

#on supprime les lignes ne contenant aucune valeur de population sous-alimentée pour obtenir un dataset 'ss-alimentation'
table_Zssal = table.copy()
index_with_nan = table_Zssal.index[table_Zssal.iloc[:,32].isnull()]
table_Zssal.drop(index_with_nan,0, inplace=True)
#on revoit à la baisse l'hypothèse de population sous-alimentée (<0.1, initiallement 0.1 trop optimiste, test avec 0.005)
#cette valeur est issue du check données sur site FAO (voir exple avec Haïti)
table_Zssal['Valeur_sousalx1e6'] = table_Zssal['Valeur_sousalx1e6'].replace(0.1 ,0.005)
#on liste les pays concernés par la ss-nutrition selon la FAO et fomattage
list_pays_ssal = table_Zssal['Zone'].unique()
# table_Zssal

#parcours du datset 'GPS' et relève des informations suivantes
List_pays = []        
List_lat  = []
List_long = []
List_cap  = []

n = len(df)
for i, row in df.iterrows():
    Zone = df.iloc[i,df.columns.get_loc('Zone')]      
#si pays contenu dans dataset 'GPS' alors récupération de ses coordonnées
    if (Zone in list_pays_ssal) : 
        pays = df.iloc[i,df.columns.get_loc('Zone')]
        List_pays.append(pays)
        capitale = df.iloc[i,df.columns.get_loc('Capitale')]
        List_cap.append(capitale)
        lat = df.iloc[i,df.columns.get_loc('Latitude')]
        List_lat.append(lat)
        long = df.iloc[i,df.columns.get_loc('Longitude')]
        List_long.append(long)

#création df avec ces Lists
df_select = pd.DataFrame({'Zone': List_pays,'Capitale': List_cap,'Latitude': List_lat,'Longitude': List_long})

#jointure du df contenant les coordonnées 'GPS' au df des pays ss-alimentés
df_vf = pd.merge(table_Zssal,df_select,how="left", on="Zone")

#par précaution, on supprime les lignes ne contenant aucune valeur de LAT ou LONG (étape normalement ok avec Zones renommées)
index_with_nan = df_vf.index[df_vf.iloc[:,35].isnull()]
df_vf.drop(index_with_nan,0, inplace=True)

C:\Users\HP\AppData\Local\Temp/ipykernel_9160/2323574714.py:6: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  table_Zssal.drop(index_with_nan,0, inplace=True)
C:\Users\HP\AppData\Local\Temp/ipykernel_9160/2323574714.py:42: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  df_vf.drop(index_with_nan,0, inplace=True)


# MAP sous-alimentation et population

In [17]:
np.zeros((300, 250, 3), dtype = int)

# Extraction des data à visualiser
lat = df_vf['Latitude'].values
lon = df_vf['Longitude'].values
population = df_vf['Valeur_populationx1000'].values*1000                     #unité millions
sous_alimentation = df_vf['Valeur_sousalx1e6'].values*1e6                    #unité millions
pourcentage = sous_alimentation/population

# création [Pourcentage] 
for i, row in df_vf.iterrows():
    try: 
        df_vf.loc[i,'Pourcentage_ssal'] = (1e6*row['Valeur_sousalx1e6'])/(1e3*row['Valeur_populationx1000'])
    except ZeroDivisionError:
        df_vf.loc[i,'Pourcentage_ssal'] = 0
        
# Préparation du background de la map
fig = plt.figure(figsize=(20, 15))
# m = Basemap(projection='cyl',llcrnrlat=-45,urcrnrlat=45,llcrnrlon=-15,urcrnrlon=60,resolution='c') # by zone
m = Basemap(projection='cyl', resolution='c', lon_0=0) # whole map
m.etopo(scale=0.5, alpha=0.5)
m.shadedrelief()
m.drawcoastlines(color='gray')
m.drawcountries(color='gray')
m.drawstates(color='gray')
       
# Echelle de couleur population par pays/capitale + taille pour sous-alimentation
m.scatter(lon,lat,latlon=True,c=np.log10(population),s=15*100*pourcentage,cmap='Reds',edgecolors = 'k',alpha=0.7)

# création colorbar + légende
# plt.colorbar(orientation="horizontal", label=r'$\log_{10}({\rm population})$')
bornm = np.log10(1000*df_vf['Valeur_populationx1000'].min())
bornp = np.log10(1000*df_vf['Valeur_populationx1000'].max())
plt.clim(bornm,bornp)

# Affectation légende avec dummy points
for pourcentage in [25, 50, 75]:
    plt.scatter([], [], c='k', edgecolors = 'k', alpha=0.5, s=15*pourcentage, label=str(pourcentage) + ' %') #marker oversizing
plt.legend(scatterpoints=1, frameon=False,labelspacing=2, loc='lower left',title='Taux de population sous-alimenté en 2013');
plt.title("Sous-alimentation dans le monde en 2013",fontsize=20)

#affichage de qlq données significtaives (nom zone)
df_vf_sample = df_vf[df_vf['Pourcentage_ssal'] > 0.40]
n=len(df_vf_sample)
for i in range (n) :
    longi = df_vf_sample.iloc[i,df_vf_sample.columns.get_loc('Longitude')] 
    lati  = df_vf_sample.iloc[i,df_vf_sample.columns.get_loc('Latitude')] 
    zo    = df_vf_sample.iloc[i,df_vf_sample.columns.get_loc('Zone')] 
    plt.plot(longi,lati, 'ok', markersize=3)
    plt.text(longi,lati, zo, fontsize=10);
    
plt.text(-20,-60,'Coloré par population', fontsize=14, color='k') 
plt.savefig('OUTPUTS_FORMATION/P3_image_ssal.jpg',dpi=300)
plt.show()

NameError: name 'Basemap' is not defined

<Figure size 1440x1080 with 0 Axes>

# MAP exportations mondiales de Blé en 2013

In [ ]:
############################ PREPARATION DU DF POUR LA TOTALITE DES PAYS ############################

#on repart de la table principale COMPLETE initiale pour afficher quelques les exportations
#on revoit à la baisse l'hypothèse de population sous-alimentée (<0.1, initiallement 0.1 trop optimiste, test avec 0.005)
table['Valeur_sousalx1e6'] = table['Valeur_sousalx1e6'].replace(0.1 ,0.005)

#on liste TOUS les pays
list_pays_all = table['Zone'].unique()

#parcours de la table et relève des informations suivantes
List_pays_all = []        
List_lat_all  = []
List_long_all = []
List_cap_all = []

n = len(df)
for i, row in df.iterrows():
    Zone = df.iloc[i,df.columns.get_loc('Zone')]      
#si pays contenu dans dataset global alors récupération de ses coordonnées
    if (Zone in list_pays_all) : 
        pays = df.iloc[i,df.columns.get_loc('Zone')]
        List_pays_all.append(pays)
        capitale = df.iloc[i,df.columns.get_loc('Capitale')]
        List_cap_all.append(capitale)
        lat = df.iloc[i,df.columns.get_loc('Latitude')]
        List_lat_all.append(lat)
        long = df.iloc[i,df.columns.get_loc('Longitude')]
        List_long_all.append(long)

#création df avec ces Lists
df_select_all = pd.DataFrame({'Zone': List_pays_all,'Capitale': List_cap_all,'Latitude': List_lat_all,'Longitude': List_long_all})
df_select_all['Zone'].unique()
#merge du df contenant les coordonnées 'GPS' au df principal
df_vf = pd.merge(table,df_select_all,how="left", on="Zone")

In [ ]:
#Soient les 4 produits les plus exportés identifiés en Q11 + quelqu'autres significatifs
list_produits =['Blé','Maïs','Lait - Excl Beurre','Soja','Manioc','Huile de Palme']
dv_vf_export = df_vf.loc[(df_vf['Produit']=='Blé') ,'Zone':'Longitude']
produit=dv_vf_export['Produit'].unique()
np.zeros((300, 250, 3), dtype = int)

# Extraction des data à visualiser
lat = dv_vf_export['Latitude'].values.astype(int)
lon = dv_vf_export['Longitude'].values
population = dv_vf_export['Valeur_populationx1000'].values*1000                 #unité millions
dv_vf_export['Exportations - Quantité'] = dv_vf_export['Exportations - Quantité'].replace(0 ,1) #pour contourner log10(0)
exportation = dv_vf_export['Exportations - Quantité'].values                    #milliers de tonne

# Préparation du background de la map
fig = plt.figure(figsize=(20, 15))
m = Basemap(projection='cyl', resolution='c', lon_0=0) # whole map
m.etopo(scale=0.5, alpha=0.5)
m.shadedrelief()
m.drawcoastlines(color='gray')
m.drawcountries(color='gray')
m.drawstates(color='gray')

# rendre échelle couleur plus lisible
norm = colors.Normalize(dv_vf_export['Exportations - Quantité'].min(), dv_vf_export['Exportations - Quantité'].max())
m.scatter(lon,lat,latlon=True,c=np.log10(exportation), norm=norm,s=exportation/8,cmap='Reds',edgecolors = 'k',alpha=0.7)

# création colorbar + légende
# plt.colorbar(orientation="horizontal", label=r'exportation')
bornm = np.log10(1)
bornp = np.log10(dv_vf_export['Exportations - Quantité'].max())
plt.clim(bornm,bornp)

#affecter légende avec dummy points 
for exportation in [1000, 3000, 5000]: #(/!\ attention au facteur d'échelle imposé dans le "m.scatter")
        plt.scatter([], [],  c='k', edgecolors = 'k', alpha=0.5, s=exportation, label=(str("      ")+str(exportation)) )
plt.legend(scatterpoints=1, frameon=False,labelspacing=5.5, loc='center left',title='Exportations (x 8000 tonnes)');
title = str('Exportations mondiales de ') + produit + str(' en 2013')
plt.title(title,fontsize=20)

#affichage de qlq données significtaives (nom zone)
dv_vf_export_sample = dv_vf_export[dv_vf_export['Exportations - Quantité']>5000]
n=len(dv_vf_export_sample)
for i in range (n) :
    longi = dv_vf_export_sample.iloc[i,dv_vf_export_sample.columns.get_loc('Longitude')] 
    lati  = dv_vf_export_sample.iloc[i,dv_vf_export_sample.columns.get_loc('Latitude')] 
    zo    = dv_vf_export_sample.iloc[i,dv_vf_export_sample.columns.get_loc('Zone')] 
    plt.plot(longi,lati, 'ok', markersize=3)
    plt.text(longi,lati, zo, fontsize=10);
    
plt.savefig('OUTPUTS_FORMATION/P3_image_exp_blé.jpg',dpi=300)
plt.show()

# MAP exportations mondiales de Maïs en 2013

In [ ]:
#Soient les 4 produits les plus exportés identifiés en Q11 + quelqu'autres significatifs
list_produits =['Blé','Maïs','Lait - Excl Beurre','Soja','Manioc','Huile de Palme']
dv_vf_export = df_vf.loc[(df_vf['Produit']=='Maïs') ,'Zone':'Longitude']
produit=dv_vf_export['Produit'].unique()
np.zeros((300, 250, 3), dtype = int)

# Extraction des data à visualiser
lat = dv_vf_export['Latitude'].values.astype(int)
lon = dv_vf_export['Longitude'].values
population = dv_vf_export['Valeur_populationx1000'].values*1000                 #unité millions
dv_vf_export['Exportations - Quantité'] = dv_vf_export['Exportations - Quantité'].replace(0 ,1) #pour contourner log10(0)
exportation = dv_vf_export['Exportations - Quantité'].values                    #milliers de tonne

# Préparation du background de la map
fig = plt.figure(figsize=(20, 15))
m = Basemap(projection='cyl', resolution='c', lon_0=0) # whole map
m.etopo(scale=0.5, alpha=0.5)
m.shadedrelief()
m.drawcoastlines(color='gray')
m.drawcountries(color='gray')
m.drawstates(color='gray')

# rendre échelle couleur plus lisible
norm = colors.Normalize(dv_vf_export['Exportations - Quantité'].min(), dv_vf_export['Exportations - Quantité'].max())
m.scatter(lon,lat,latlon=True,c=np.log10(exportation), norm=norm,s=exportation/8,cmap='Reds',edgecolors = 'k',alpha=0.7)

# création colorbar + légende
# plt.colorbar(orientation="horizontal", label=r'exportation')
bornm = np.log10(1)
bornp = np.log10(dv_vf_export['Exportations - Quantité'].max())
plt.clim(bornm,bornp)

#affecter légende avec dummy points 
for exportation in [1000, 3000, 5000]: #(/!\ attention au facteur d'échelle imposé dans le "m.scatter")
        plt.scatter([], [],  c='k', edgecolors = 'k', alpha=0.5, s=exportation, label=(str("      ")+str(exportation)) )
plt.legend(scatterpoints=1, frameon=False,labelspacing=5.5, loc='center left',title='Exportations (x 8000 tonnes)');
title = str('Exportations mondiales de ') + produit + str(' en 2013')
plt.title(title,fontsize=20)

#affichage de qlq données significtaives (nom zone)
dv_vf_export_sample = dv_vf_export[dv_vf_export['Exportations - Quantité']>5000]
n=len(dv_vf_export_sample)
for i in range (n) :
    longi = dv_vf_export_sample.iloc[i,dv_vf_export_sample.columns.get_loc('Longitude')] 
    lati  = dv_vf_export_sample.iloc[i,dv_vf_export_sample.columns.get_loc('Latitude')] 
    zo    = dv_vf_export_sample.iloc[i,dv_vf_export_sample.columns.get_loc('Zone')] 
    plt.plot(longi,lati, 'ok', markersize=3)
    plt.text(longi,lati, zo, fontsize=10);
    
plt.savefig('OUTPUTS_FORMATION/P3_image_exp_maïs.jpg',dpi=300)
plt.show()

# MAP exportations mondiales de Lait en 2013

In [ ]:
#Soient les 4 produits les plus exportés identifiés en Q11 + quelqu'autres significatifs
list_produits =['Blé','Maïs','Lait - Excl Beurre','Soja','Manioc','Huile de Palme']
dv_vf_export = df_vf.loc[(df_vf['Produit']=='Lait - Excl Beurre') ,'Zone':'Longitude']
produit=dv_vf_export['Produit'].unique()
np.zeros((300, 250, 3), dtype = int)

# Extraction des data à visualiser
lat = dv_vf_export['Latitude'].values.astype(int)
lon = dv_vf_export['Longitude'].values
population = dv_vf_export['Valeur_populationx1000'].values*1000                 #unité millions
dv_vf_export['Exportations - Quantité'] = dv_vf_export['Exportations - Quantité'].replace(0 ,1) #pour contourner log10(0)
exportation = dv_vf_export['Exportations - Quantité'].values                    #milliers de tonne

# Préparation du background de la map
fig = plt.figure(figsize=(20, 15))
m = Basemap(projection='cyl', resolution='c', lon_0=0) # whole map
m.etopo(scale=0.5, alpha=0.5)
m.shadedrelief()
m.drawcoastlines(color='gray')
m.drawcountries(color='gray')
m.drawstates(color='gray')

# rendre échelle couleur plus lisible
norm = colors.Normalize(dv_vf_export['Exportations - Quantité'].min(), dv_vf_export['Exportations - Quantité'].max())
m.scatter(lon,lat,latlon=True,c=np.log10(exportation), norm=norm,s=exportation/8,cmap='Reds',edgecolors = 'k',alpha=0.7)

# création colorbar + légende
# plt.colorbar(orientation="horizontal", label=r'exportation')
bornm = np.log10(1)
bornp = np.log10(dv_vf_export['Exportations - Quantité'].max())
plt.clim(bornm,bornp)

#affecter légende avec dummy points 
for exportation in [1000, 3000, 5000]: #(/!\ attention au facteur d'échelle imposé dans le "m.scatter")
        plt.scatter([], [],  c='k', edgecolors = 'k', alpha=0.5, s=exportation, label=(str("      ")+str(exportation)) )
plt.legend(scatterpoints=1, frameon=False,labelspacing=5.5, loc='center left',title='Exportations (x 8000 tonnes)');
title = str('Exportations mondiales de ') + produit + str(' en 2013')
plt.title(title,fontsize=20)

#affichage de qlq données significtaives (nom zone)
dv_vf_export_sample = dv_vf_export[dv_vf_export['Exportations - Quantité']>5000]
n=len(dv_vf_export_sample)
for i in range (n) :
    longi = dv_vf_export_sample.iloc[i,dv_vf_export_sample.columns.get_loc('Longitude')] 
    lati  = dv_vf_export_sample.iloc[i,dv_vf_export_sample.columns.get_loc('Latitude')] 
    zo    = dv_vf_export_sample.iloc[i,dv_vf_export_sample.columns.get_loc('Zone')] 
    plt.plot(longi,lati, 'ok', markersize=3)
    plt.text(longi,lati, zo, fontsize=10);
    
plt.savefig('OUTPUTS_FORMATION/P3_image_exp_lait.jpg',dpi=300)
plt.show()

# MAP exportations mondiales de Soja en 2013

In [ ]:
#Soient les 4 produits les plus exportés identifiés en Q11 + quelqu'autres significatifs
list_produits =['Blé','Maïs','Lait - Excl Beurre','Soja','Manioc','Huile de Palme']
dv_vf_export = df_vf.loc[(df_vf['Produit']=='Soja') ,'Zone':'Longitude']
produit=dv_vf_export['Produit'].unique()
np.zeros((300, 250, 3), dtype = int)

# Extraction des data à visualiser
lat = dv_vf_export['Latitude'].values.astype(int)
lon = dv_vf_export['Longitude'].values
population = dv_vf_export['Valeur_populationx1000'].values*1000                 #unité millions
dv_vf_export['Exportations - Quantité'] = dv_vf_export['Exportations - Quantité'].replace(0 ,1) #pour contourner log10(0)
exportation = dv_vf_export['Exportations - Quantité'].values                    #milliers de tonne

# Préparation du background de la map
fig = plt.figure(figsize=(20, 15))
m = Basemap(projection='cyl', resolution='c', lon_0=0) # whole map
m.etopo(scale=0.5, alpha=0.5)
m.shadedrelief()
m.drawcoastlines(color='gray')
m.drawcountries(color='gray')
m.drawstates(color='gray')

# rendre échelle couleur plus lisible
norm = colors.Normalize(dv_vf_export['Exportations - Quantité'].min(), dv_vf_export['Exportations - Quantité'].max())
m.scatter(lon,lat,latlon=True,c=np.log10(exportation), norm=norm,s=exportation/8,cmap='Reds',edgecolors = 'k',alpha=0.7)

# création colorbar + légende
# plt.colorbar(orientation="horizontal", label=r'exportation')
bornm = np.log10(1)
bornp = np.log10(dv_vf_export['Exportations - Quantité'].max())
plt.clim(bornm,bornp)

#affecter légende avec dummy points 
for exportation in [1000, 3000, 5000]: #(/!\ attention au facteur d'échelle imposé dans le "m.scatter")
        plt.scatter([], [],  c='k', edgecolors = 'k', alpha=0.5, s=exportation, label=(str("      ")+str(exportation)) )
plt.legend(scatterpoints=1, frameon=False,labelspacing=5.5, loc='center left',title='Exportations (x 8000 tonnes)');
title = str('Exportations mondiales de ') + produit + str(' en 2013')
plt.title(title,fontsize=20)

#affichage de qlq données significtaives (nom zone)
dv_vf_export_sample = dv_vf_export[dv_vf_export['Exportations - Quantité']>5000]
n=len(dv_vf_export_sample)
for i in range (n) :
    longi = dv_vf_export_sample.iloc[i,dv_vf_export_sample.columns.get_loc('Longitude')] 
    lati  = dv_vf_export_sample.iloc[i,dv_vf_export_sample.columns.get_loc('Latitude')] 
    zo    = dv_vf_export_sample.iloc[i,dv_vf_export_sample.columns.get_loc('Zone')] 
    plt.plot(longi,lati, 'ok', markersize=3)
    plt.text(longi,lati, zo, fontsize=10);
    
plt.savefig('OUTPUTS_FORMATION/P3_image_exp_soja.jpg',dpi=300)
plt.show()

# MAP exportations mondiales de Manioc en 2013

In [ ]:
#Soient les 4 produits les plus exportés identifiés en Q11 + quelqu'autres significatifs
list_produits =['Blé','Maïs','Lait - Excl Beurre','Soja','Manioc','Huile de Palme']
dv_vf_export = df_vf.loc[(df_vf['Produit']=='Manioc') ,'Zone':'Longitude']
produit=dv_vf_export['Produit'].unique()
np.zeros((300, 250, 3), dtype = int)

# Extraction des data à visualiser
lat = dv_vf_export['Latitude'].values.astype(int)
lon = dv_vf_export['Longitude'].values
population = dv_vf_export['Valeur_populationx1000'].values*1000                 #unité millions
dv_vf_export['Exportations - Quantité'] = dv_vf_export['Exportations - Quantité'].replace(0 ,1) #pour contourner log10(0)
exportation = dv_vf_export['Exportations - Quantité'].values                    #milliers de tonne

# Préparation du background de la map
fig = plt.figure(figsize=(20, 15))
m = Basemap(projection='cyl', resolution='c', lon_0=0) # whole map
m.etopo(scale=0.5, alpha=0.5)
m.shadedrelief()
m.drawcoastlines(color='gray')
m.drawcountries(color='gray')
m.drawstates(color='gray')

# rendre échelle couleur plus lisible
norm = colors.Normalize(dv_vf_export['Exportations - Quantité'].min(), dv_vf_export['Exportations - Quantité'].max())
m.scatter(lon,lat,latlon=True,c=np.log10(exportation), norm=norm,s=exportation/8,cmap='Reds',edgecolors = 'k',alpha=0.7)

# création colorbar + légende
# plt.colorbar(orientation="horizontal", label=r'exportation')
bornm = np.log10(1)
bornp = np.log10(dv_vf_export['Exportations - Quantité'].max())
plt.clim(bornm,bornp)

#affecter légende avec dummy points 
for exportation in [1000, 3000, 5000]: #(/!\ attention au facteur d'échelle imposé dans le "m.scatter")
        plt.scatter([], [],  c='k', edgecolors = 'k', alpha=0.5, s=exportation, label=(str("      ")+str(exportation)) )
plt.legend(scatterpoints=1, frameon=False,labelspacing=5.5, loc='center left',title='Exportations (x 8000 tonnes)');
title = str('Exportations mondiales de ') + produit + str(' en 2013')
plt.title(title,fontsize=20)

#affichage de qlq données significtaives (nom zone)
dv_vf_export_sample = dv_vf_export[dv_vf_export['Exportations - Quantité']>5000]
n=len(dv_vf_export_sample)
for i in range (n) :
    longi = dv_vf_export_sample.iloc[i,dv_vf_export_sample.columns.get_loc('Longitude')] 
    lati  = dv_vf_export_sample.iloc[i,dv_vf_export_sample.columns.get_loc('Latitude')] 
    zo    = dv_vf_export_sample.iloc[i,dv_vf_export_sample.columns.get_loc('Zone')] 
    plt.plot(longi,lati, 'ok', markersize=3)
    plt.text(longi,lati, zo, fontsize=10);
    
plt.savefig('OUTPUTS_FORMATION/P3_image_exp_manioc.jpg',dpi=300)
plt.show()

# MAP exportations mondiales d'Huile de Palme en 2013

In [ ]:
#Soient les 4 produits les plus exportés identifiés en Q11 + quelqu'autres significatifs
list_produits =['Blé','Maïs','Lait - Excl Beurre','Soja','Manioc','Huile de Palme']
dv_vf_export = df_vf.loc[(df_vf['Produit']=='Huile de Palme') ,'Zone':'Longitude']
produit=dv_vf_export['Produit'].unique()
np.zeros((300, 250, 3), dtype = int)

# Extraction des data à visualiser
lat = dv_vf_export['Latitude'].values.astype(int)
lon = dv_vf_export['Longitude'].values
population = dv_vf_export['Valeur_populationx1000'].values*1000                 #unité millions
dv_vf_export['Exportations - Quantité'] = dv_vf_export['Exportations - Quantité'].replace(0 ,1) #pour contourner log10(0)
exportation = dv_vf_export['Exportations - Quantité'].values                    #milliers de tonne

# Préparation du background de la map
fig = plt.figure(figsize=(20, 15))
m = Basemap(projection='cyl', resolution='c', lon_0=0) # whole map
m.etopo(scale=0.5, alpha=0.5)
m.shadedrelief()
m.drawcoastlines(color='gray')
m.drawcountries(color='gray')
m.drawstates(color='gray')

# rendre échelle couleur plus lisible
norm = colors.Normalize(dv_vf_export['Exportations - Quantité'].min(), dv_vf_export['Exportations - Quantité'].max())
m.scatter(lon,lat,latlon=True,c=np.log10(exportation), norm=norm,s=exportation/8,cmap='Reds',edgecolors = 'k',alpha=0.7)

# création colorbar + légende
# plt.colorbar(orientation="horizontal", label=r'exportation')
bornm = np.log10(1)
bornp = np.log10(dv_vf_export['Exportations - Quantité'].max())
plt.clim(bornm,bornp)

#affecter légende avec dummy points 
for exportation in [1000, 3000, 5000]: #(/!\ attention au facteur d'échelle imposé dans le "m.scatter")
        plt.scatter([], [],  c='k', edgecolors = 'k', alpha=0.5, s=exportation, label=(str("      ")+str(exportation)) )
plt.legend(scatterpoints=1, frameon=False,labelspacing=5.5, loc='center left',title='Exportations (x 8000 tonnes)');
title = str('Exportations mondiales de ') + produit + str(' en 2013')
plt.title(title,fontsize=20)

#affichage de qlq données significtaives (nom zone)
dv_vf_export_sample = dv_vf_export[dv_vf_export['Exportations - Quantité']>5000]
n=len(dv_vf_export_sample)
for i in range (n) :
    longi = dv_vf_export_sample.iloc[i,dv_vf_export_sample.columns.get_loc('Longitude')] 
    lati  = dv_vf_export_sample.iloc[i,dv_vf_export_sample.columns.get_loc('Latitude')] 
    zo    = dv_vf_export_sample.iloc[i,dv_vf_export_sample.columns.get_loc('Zone')] 
    plt.plot(longi,lati, 'ok', markersize=3)
    plt.text(longi,lati, zo, fontsize=10);
    
plt.savefig('OUTPUTS_FORMATION/P3_image_exp_huile_de_palme.jpg',dpi=300)
plt.show()

# MAP disponibilité intérieure de Blé en 2013

In [ ]:
#Soient les 4 produits les plus exportés identifiés en Q11 + quelqu'autres significatifs
list_produits =['Blé','Maïs','Lait - Excl Beurre','Soja','Manioc','Huile de Palme']
dv_vf_dispo = df_vf.loc[(df_vf['Produit']=='Blé') ,'Zone':'Longitude']
produit=dv_vf_dispo['Produit'].unique()
np.zeros((300, 250, 3), dtype = int)

# Extraction des data à visualiser
lat = dv_vf_dispo['Latitude'].values.astype(int)
lon = dv_vf_dispo['Longitude'].values
dispo_int = dv_vf_dispo['Disponibilité intérieure'].values                    #milliers de tonne
population = dv_vf_dispo['Valeur_populationx1000'].values*1000                 #unité millions
exportation = dv_vf_dispo['Exportations - Quantité'].values                     #milliers de tonne
dv_vf_dispo['Importations - Quantité'] = dv_vf_dispo['Importations - Quantité'].replace(0 ,1) #pour contourner log10(0)
importation = dv_vf_dispo['Importations - Quantité'].values                     #milliers de tonne

# préparation du background de la map
fig = plt.figure(figsize=(20, 15))
m = Basemap(projection='cyl', resolution='c', lon_0=0) # whole map
m.etopo(scale=0.5, alpha=0.5)
m.shadedrelief()
m.drawcoastlines(color='gray')
m.drawcountries(color='gray')
m.drawstates(color='gray')

norm = colors.Normalize(dispo_int.min(), dispo_int.max())
m.scatter(lon,lat,latlon=True,c=np.log10(importation), norm=norm,s=dispo_int/20,cmap='Reds',edgecolors = 'k',alpha=0.7)

# création colorbar + légende
# plt.colorbar(orientation="horizontal", label=r'$\log_{10}({\rm importation})$')
bornm = np.log10(1)
bornp = np.log10(dv_vf_dispo['Importations - Quantité'].max())
plt.clim(bornm,bornp)

#affecter légende avec dummy points
for dispo_int in [25000, 50000, 100000]: #(/!\ attention au facteur d'échelle imposé dans le "m.scatter")
        plt.scatter([], [],  c='k', edgecolors = 'k', alpha=0.5, s=dispo_int/20, label=(str("      ")+str(dispo_int)))
plt.legend(scatterpoints=1, frameon=False,labelspacing=5.5, loc='center left',title='Disponibilité intérieure (tonnes)');
title = str('Disponibilité intérieure pour le ') + produit + str(' en 2013')
plt.title(title,fontsize=20)

#affichage de qlq données significtaives (nom zone)
dv_vf_dispo_sample = dv_vf_dispo[dv_vf_dispo['Disponibilité intérieure']>20000]
n=len(dv_vf_dispo_sample)
for i in range (n) :
    longi = dv_vf_dispo_sample.iloc[i,dv_vf_dispo_sample.columns.get_loc('Longitude')] 
    lati  = dv_vf_dispo_sample.iloc[i,dv_vf_dispo_sample.columns.get_loc('Latitude')] 
    zo    = dv_vf_dispo_sample.iloc[i,dv_vf_dispo_sample.columns.get_loc('Zone')] 
    plt.plot(longi,lati, 'ok', markersize=3)
    plt.text(longi,lati, zo, fontsize=10);

plt.text(-20,-60,'Coloré par importation absolue', fontsize=10, color='k') 
plt.savefig('OUTPUTS_FORMATION/P3_image_dispo_int_blé.jpg',dpi=300)
plt.show()

# MAP disponibilité intérieure de Maïs en 2013

In [ ]:
#Soient les 4 produits les plus exportés identifiés en Q11 + quelqu'autres significatifs
list_produits =['Blé','Maïs','Lait - Excl Beurre','Soja','Manioc','Huile de Palme']
dv_vf_dispo = df_vf.loc[(df_vf['Produit']=='Maïs') ,'Zone':'Longitude']
produit=dv_vf_dispo['Produit'].unique()
np.zeros((300, 250, 3), dtype = int)

# Extraction des data à visualiser
lat = dv_vf_dispo['Latitude'].values.astype(int)
lon = dv_vf_dispo['Longitude'].values
dispo_int = dv_vf_dispo['Disponibilité intérieure'].values                    #milliers de tonne
population = dv_vf_dispo['Valeur_populationx1000'].values*1000                 #unité millions
exportation = dv_vf_dispo['Exportations - Quantité'].values                     #milliers de tonne
dv_vf_dispo['Importations - Quantité'] = dv_vf_dispo['Importations - Quantité'].replace(0 ,1) #pour contourner log10(0)
importation = dv_vf_dispo['Importations - Quantité'].values                     #milliers de tonne

# préparation du background de la map
fig = plt.figure(figsize=(20, 15))
m = Basemap(projection='cyl', resolution='c', lon_0=0) # whole map
m.etopo(scale=0.5, alpha=0.5)
m.shadedrelief()
m.drawcoastlines(color='gray')
m.drawcountries(color='gray')
m.drawstates(color='gray')

norm = colors.Normalize(dispo_int.min(), dispo_int.max())
m.scatter(lon,lat,latlon=True,c=np.log10(importation), norm=norm,s=dispo_int/20,cmap='Reds',edgecolors = 'k',alpha=0.7)

# création colorbar + légende
# plt.colorbar(orientation="horizontal", label=r'$\log_{10}({\rm importation})$')
bornm = np.log10(1)
bornp = np.log10(dv_vf_dispo['Importations - Quantité'].max())
plt.clim(bornm,bornp)

#affecter légende avec dummy points
for dispo_int in [25000, 50000, 100000]: #(/!\ attention au facteur d'échelle imposé dans le "m.scatter")
        plt.scatter([], [],  c='k', edgecolors = 'k', alpha=0.5, s=dispo_int/20, label=(str("      ")+str(dispo_int)))
plt.legend(scatterpoints=1, frameon=False,labelspacing=5.5, loc='center left',title='Disponibilité intérieure (tonnes)');
title = str('Disponibilité intérieure pour le ') + produit + str(' en 2013')
plt.title(title,fontsize=20)

#affichage de qlq données significtaives (nom zone)
dv_vf_dispo_sample = dv_vf_dispo[dv_vf_dispo['Disponibilité intérieure']>20000]
n=len(dv_vf_dispo_sample)
for i in range (n) :
    longi = dv_vf_dispo_sample.iloc[i,dv_vf_dispo_sample.columns.get_loc('Longitude')] 
    lati  = dv_vf_dispo_sample.iloc[i,dv_vf_dispo_sample.columns.get_loc('Latitude')] 
    zo    = dv_vf_dispo_sample.iloc[i,dv_vf_dispo_sample.columns.get_loc('Zone')] 
    plt.plot(longi,lati, 'ok', markersize=3)
    plt.text(longi,lati, zo, fontsize=10);

plt.text(-20,-60,'Coloré par importation absolue', fontsize=10, color='k') 
plt.savefig('OUTPUTS_FORMATION/P3_image_dispo_int_maïs.jpg',dpi=300)
plt.show()

# MAP disponibilité intérieure de Lait en 2013

In [ ]:
#Soient les 4 produits les plus exportés identifiés en Q11 + quelqu'autres significatifs
list_produits =['Blé','Maïs','Lait - Excl Beurre','Soja','Manioc','Huile de Palme']
dv_vf_dispo = df_vf.loc[(df_vf['Produit']=='Lait - Excl Beurre') ,'Zone':'Longitude']
produit=dv_vf_dispo['Produit'].unique()
np.zeros((300, 250, 3), dtype = int)

# Extraction des data à visualiser
lat = dv_vf_dispo['Latitude'].values.astype(int)
lon = dv_vf_dispo['Longitude'].values
dispo_int = dv_vf_dispo['Disponibilité intérieure'].values                    #milliers de tonne
population = dv_vf_dispo['Valeur_populationx1000'].values*1000                 #unité millions
exportation = dv_vf_dispo['Exportations - Quantité'].values                     #milliers de tonne
dv_vf_dispo['Importations - Quantité'] = dv_vf_dispo['Importations - Quantité'].replace(0 ,1) #pour contourner log10(0)
importation = dv_vf_dispo['Importations - Quantité'].values                     #milliers de tonne

# préparation du background de la map
fig = plt.figure(figsize=(20, 15))
m = Basemap(projection='cyl', resolution='c', lon_0=0) # whole map
m.etopo(scale=0.5, alpha=0.5)
m.shadedrelief()
m.drawcoastlines(color='gray')
m.drawcountries(color='gray')
m.drawstates(color='gray')

norm = colors.Normalize(dispo_int.min(), dispo_int.max())
m.scatter(lon,lat,latlon=True,c=np.log10(importation), norm=norm,s=dispo_int/20,cmap='Reds',edgecolors = 'k',alpha=0.7)

# création colorbar + légende
# plt.colorbar(orientation="horizontal", label=r'$\log_{10}({\rm importation})$')
bornm = np.log10(1)
bornp = np.log10(dv_vf_dispo['Importations - Quantité'].max())
plt.clim(bornm,bornp)

#affecter légende avec dummy points
for dispo_int in [25000, 50000, 100000]: #(/!\ attention au facteur d'échelle imposé dans le "m.scatter")
        plt.scatter([], [],  c='k', edgecolors = 'k', alpha=0.5, s=dispo_int/20, label=(str("      ")+str(dispo_int)))
plt.legend(scatterpoints=1, frameon=False,labelspacing=5.5, loc='center left',title='Disponibilité intérieure (tonnes)');
title = str('Disponibilité intérieure pour le ') + produit + str(' en 2013')
plt.title(title,fontsize=20)

#affichage de qlq données significtaives (nom zone)
dv_vf_dispo_sample = dv_vf_dispo[dv_vf_dispo['Disponibilité intérieure']>20000]
n=len(dv_vf_dispo_sample)
for i in range (n) :
    longi = dv_vf_dispo_sample.iloc[i,dv_vf_dispo_sample.columns.get_loc('Longitude')] 
    lati  = dv_vf_dispo_sample.iloc[i,dv_vf_dispo_sample.columns.get_loc('Latitude')] 
    zo    = dv_vf_dispo_sample.iloc[i,dv_vf_dispo_sample.columns.get_loc('Zone')] 
    plt.plot(longi,lati, 'ok', markersize=3)
    plt.text(longi,lati, zo, fontsize=10);

plt.text(-20,-60,'Coloré par importation absolue', fontsize=10, color='k') 
plt.savefig('OUTPUTS_FORMATION/P3_image_dispo_int_lait.jpg',dpi=300)
plt.show()

# MAP disponibilité intérieure de Soja en 2013

In [ ]:
#Soient les 4 produits les plus exportés identifiés en Q11 + quelqu'autres significatifs
list_produits =['Blé','Maïs','Lait - Excl Beurre','Soja','Manioc','Huile de Palme']
dv_vf_dispo = df_vf.loc[(df_vf['Produit']=='Soja') ,'Zone':'Longitude']
produit=dv_vf_dispo['Produit'].unique()
np.zeros((300, 250, 3), dtype = int)

# Extraction des data à visualiser
lat = dv_vf_dispo['Latitude'].values.astype(int)
lon = dv_vf_dispo['Longitude'].values
dispo_int = dv_vf_dispo['Disponibilité intérieure'].values                    #milliers de tonne
population = dv_vf_dispo['Valeur_populationx1000'].values*1000                 #unité millions
exportation = dv_vf_dispo['Exportations - Quantité'].values                     #milliers de tonne
dv_vf_dispo['Importations - Quantité'] = dv_vf_dispo['Importations - Quantité'].replace(0 ,1) #pour contourner log10(0)
importation = dv_vf_dispo['Importations - Quantité'].values                     #milliers de tonne

# préparation du background de la map
fig = plt.figure(figsize=(20, 15))
m = Basemap(projection='cyl', resolution='c', lon_0=0) # whole map
m.etopo(scale=0.5, alpha=0.5)
m.shadedrelief()
m.drawcoastlines(color='gray')
m.drawcountries(color='gray')
m.drawstates(color='gray')

norm = colors.Normalize(dispo_int.min(), dispo_int.max())
m.scatter(lon,lat,latlon=True,c=np.log10(importation), norm=norm,s=dispo_int/20,cmap='Reds',edgecolors = 'k',alpha=0.7)

# création colorbar + légende
# plt.colorbar(orientation="horizontal", label=r'$\log_{10}({\rm importation})$')
bornm = np.log10(1)
bornp = np.log10(dv_vf_dispo['Importations - Quantité'].max())
plt.clim(bornm,bornp)

#affecter légende avec dummy points
for dispo_int in [25000, 50000, 100000]: #(/!\ attention au facteur d'échelle imposé dans le "m.scatter")
        plt.scatter([], [],  c='k', edgecolors = 'k', alpha=0.5, s=dispo_int/20, label=(str("      ")+str(dispo_int)))
plt.legend(scatterpoints=1, frameon=False,labelspacing=5.5, loc='center left',title='Disponibilité intérieure (tonnes)');
title = str('Disponibilité intérieure pour le ') + produit + str(' en 2013')
plt.title(title,fontsize=20)

#affichage de qlq données significtaives (nom zone)
dv_vf_dispo_sample = dv_vf_dispo[dv_vf_dispo['Disponibilité intérieure']>10000]
n=len(dv_vf_dispo_sample)
for i in range (n) :
    longi = dv_vf_dispo_sample.iloc[i,dv_vf_dispo_sample.columns.get_loc('Longitude')] 
    lati  = dv_vf_dispo_sample.iloc[i,dv_vf_dispo_sample.columns.get_loc('Latitude')] 
    zo    = dv_vf_dispo_sample.iloc[i,dv_vf_dispo_sample.columns.get_loc('Zone')] 
    plt.plot(longi,lati, 'ok', markersize=3)
    plt.text(longi,lati, zo, fontsize=10);

plt.text(-20,-60,'Coloré par importation absolue', fontsize=10, color='k') 
plt.savefig('OUTPUTS_FORMATION/P3_image_dispo_int_soja.jpg',dpi=300)
plt.show()

# MAP disponibilité intérieure de Manioc en 2013

In [ ]:
#Soient les 4 produits les plus exportés identifiés en Q11 + quelqu'autres significatifs
list_produits =['Blé','Maïs','Lait - Excl Beurre','Soja','Manioc','Huile de Palme']
dv_vf_dispo = df_vf.loc[(df_vf['Produit']=='Manioc') ,'Zone':'Longitude']
produit=dv_vf_dispo['Produit'].unique()
np.zeros((300, 250, 3), dtype = int)

# Extraction des data à visualiser
lat = dv_vf_dispo['Latitude'].values.astype(int)
lon = dv_vf_dispo['Longitude'].values
dispo_int = dv_vf_dispo['Disponibilité intérieure'].values                    #milliers de tonne
population = dv_vf_dispo['Valeur_populationx1000'].values*1000                 #unité millions
exportation = dv_vf_dispo['Exportations - Quantité'].values                     #milliers de tonne
dv_vf_dispo['Importations - Quantité'] = dv_vf_dispo['Importations - Quantité'].replace(0 ,1) #pour contourner log10(0)
importation = dv_vf_dispo['Importations - Quantité'].values                     #milliers de tonne

# préparation du background de la map
fig = plt.figure(figsize=(20, 15))
m = Basemap(projection='cyl', resolution='c', lon_0=0) # whole map
m.etopo(scale=0.5, alpha=0.5)
m.shadedrelief()
m.drawcoastlines(color='gray')
m.drawcountries(color='gray')
m.drawstates(color='gray')

norm = colors.Normalize(dispo_int.min(), dispo_int.max())
m.scatter(lon,lat,latlon=True,c=np.log10(importation), norm=norm,s=dispo_int/10,cmap='Reds',edgecolors = 'k',alpha=0.7)

# création colorbar + légende
# plt.colorbar(orientation="horizontal", label=r'$\log_{10}({\rm importation})$')
bornm = np.log10(1)
bornp = np.log10(dv_vf_dispo['Importations - Quantité'].max())
plt.clim(bornm,bornp)

#affecter légende avec dummy points
for dispo_int in [12500, 25000, 50000]: #(/!\ attention au facteur d'échelle imposé dans le "m.scatter")
        plt.scatter([], [],  c='k', edgecolors = 'k', alpha=0.5, s=dispo_int/10, label=(str("      ")+str(dispo_int)))
plt.legend(scatterpoints=1, frameon=False,labelspacing=5.5, loc='center left',title='Disponibilité intérieure (tonnes)');
title = str('Disponibilité intérieure pour le ') + produit + str(' en 2013')
plt.title(title,fontsize=20)

#affichage de qlq données significtaives (nom zone)
dv_vf_dispo_sample = dv_vf_dispo[dv_vf_dispo['Disponibilité intérieure']>5000]
n=len(dv_vf_dispo_sample)
for i in range (n) :
    longi = dv_vf_dispo_sample.iloc[i,dv_vf_dispo_sample.columns.get_loc('Longitude')] 
    lati  = dv_vf_dispo_sample.iloc[i,dv_vf_dispo_sample.columns.get_loc('Latitude')] 
    zo    = dv_vf_dispo_sample.iloc[i,dv_vf_dispo_sample.columns.get_loc('Zone')] 
    plt.plot(longi,lati, 'ok', markersize=3)
    plt.text(longi,lati, zo, fontsize=10);

plt.text(-20,-60,'Coloré par importation absolue', fontsize=10, color='k')    
plt.savefig('OUTPUTS_FORMATION/P3_image_dispo_int_manioc.jpg',dpi=300)
plt.show()

# MAP disponibilité intérieure de Huile de Palme en 2013

In [ ]:
#Soient les 4 produits les plus exportés identifiés en Q11 + quelqu'autres significatifs
list_produits =['Blé','Maïs','Lait - Excl Beurre','Soja','Manioc','Huile de Palme']
dv_vf_dispo = df_vf.loc[(df_vf['Produit']=='Huile de Palme') ,'Zone':'Longitude']
produit=dv_vf_dispo['Produit'].unique()
np.zeros((300, 250, 3), dtype = int)

# Extraction des data à visualiser
lat = dv_vf_dispo['Latitude'].values.astype(int)
lon = dv_vf_dispo['Longitude'].values
dispo_int = dv_vf_dispo['Disponibilité intérieure'].values                    #milliers de tonne
population = dv_vf_dispo['Valeur_populationx1000'].values*1000                 #unité millions
exportation = dv_vf_dispo['Exportations - Quantité'].values                     #milliers de tonne
dv_vf_dispo['Importations - Quantité'] = dv_vf_dispo['Importations - Quantité'].replace(0 ,1) #pour contourner log10(0)
importation = dv_vf_dispo['Importations - Quantité'].values                     #milliers de tonne

# préparation du background de la map
fig = plt.figure(figsize=(20, 15))
m = Basemap(projection='cyl', resolution='c', lon_0=0) # whole map
m.etopo(scale=0.5, alpha=0.5)
m.shadedrelief()
m.drawcoastlines(color='gray')
m.drawcountries(color='gray')
m.drawstates(color='gray')

norm = colors.Normalize(dispo_int.min(), dispo_int.max())
m.scatter(lon,lat,latlon=True,c=np.log10(importation), norm=norm,s=dispo_int/2,cmap='Reds',edgecolors = 'k',alpha=0.7)

# création colorbar + légende
# plt.colorbar(orientation="horizontal", label=r'$\log_{10}({\rm importation})$')
bornm = np.log10(1)
bornp = np.log10(dv_vf_dispo['Importations - Quantité'].max())
plt.clim(bornm,bornp)

#affecter légende avec dummy points
for dispo_int in [1000, 5000, 9000]: #(/!\ attention au facteur d'échelle imposé dans le "m.scatter")
        plt.scatter([], [],  c='k', edgecolors = 'k', alpha=0.5, s=dispo_int/2, label=(str("      ")+str(dispo_int)))
plt.legend(scatterpoints=1, frameon=False,labelspacing=5.5, loc='center left',title='Disponibilité intérieure (tonnes)');
title = str('Disponibilité intérieure pour le ') + produit + str(' en 2013')
plt.title(title,fontsize=20)

#affichage de qlq données significtaives (nom zone)
dv_vf_dispo_sample = dv_vf_dispo[dv_vf_dispo['Disponibilité intérieure']>1000]
n=len(dv_vf_dispo_sample)
for i in range (n) :
    longi = dv_vf_dispo_sample.iloc[i,dv_vf_dispo_sample.columns.get_loc('Longitude')] 
    lati  = dv_vf_dispo_sample.iloc[i,dv_vf_dispo_sample.columns.get_loc('Latitude')] 
    zo    = dv_vf_dispo_sample.iloc[i,dv_vf_dispo_sample.columns.get_loc('Zone')] 
    plt.plot(longi,lati, 'ok', markersize=3)
    plt.text(longi,lati, zo, fontsize=10);

plt.text(-20,-60,'Coloré par importation absolue', fontsize=10, color='k')    
plt.savefig('OUTPUTS_FORMATION/P3_image_dispo_int_huile_de_palme.jpg',dpi=300)
plt.show()

# Ratio Exportations / Quantité pour utilisation intérieure de Manioc en 2013

In [ ]:
#Soient les 4 produits les plus exportés identifiés en Q11 + quelqu'autres significatifs
list_produits =['Blé','Maïs','Lait - Excl Beurre','Soja','Manioc','Huile de Palme']
dv_vf_dispo = df_vf.loc[(df_vf['Produit']=='Manioc') ,'Zone':'Longitude']
produit=dv_vf_dispo['Produit'].unique()
np.zeros((300, 250, 3), dtype = int)

# Extraction des data à visualiser
lat = dv_vf_dispo['Latitude'].values.astype(int)
lon = dv_vf_dispo['Longitude'].values
dispo_int   = dv_vf_dispo['Disponibilité intérieure'].values 
population  = dv_vf_dispo['Valeur_populationx1000'].values*1000                 #unité millions
production  = dv_vf_dispo['Production'].values                                  #milliers de tonne
exportation = dv_vf_dispo['Exportations - Quantité'].values                     #milliers de tonne
importation = dv_vf_dispo['Importations - Quantité'].values                     #milliers de tonne
varstock    = dv_vf_dispo['Variation de stock'].values                          #milliers de tonne
pourcentage = exportation/(exportation+dispo_int)

# création [Ratio] 
for i, row in dv_vf_dispo.iterrows():
    try: 
        dv_vf_dispo.loc[i,'Ratio_export_dispo'] = row['Exportations - Quantité']/(row['Exportations - Quantité']+row['Disponibilité intérieure'])
    except ZeroDivisionError:
        dv_vf_dispo.loc[i,'Ratio_export_dispo'] = 0
        
# préparation du background de la map
fig = plt.figure(figsize=(20, 15))
# m = Basemap(projection='cyl',llcrnrlat=-45,urcrnrlat=45,llcrnrlon=-15,urcrnrlon=60,resolution='c') # by zone
m = Basemap(projection='cyl', resolution='c', lon_0=0) # whole map
m.etopo(scale=0.5, alpha=0.5)
m.shadedrelief()
m.drawcoastlines(color='gray')
m.drawcountries(color='gray')
m.drawstates(color='gray')
       
# Echelle de couleur population par pays/capitale + taille pour sous-alimentation
m.scatter(lon,lat,latlon=True,c=np.log10(exportation),s=20*100*pourcentage,cmap='Reds',edgecolors = 'k',alpha=0.7)

# création colorbar + légende
# plt.colorbar(orientation="horizontal", label=r'$\log_{10}({\rm exportation})$')
bornm = np.log10(1)
bornp = np.log10(df_vf['Exportations - Quantité'].max())
plt.clim(bornm,bornp)

#affecter légende avec dummy points
for pourcentage in [33, 66, 100]: #(/!\ attention au facteur d'échelle imposé dans le "m.scatter")
    plt.scatter([], [],  c='k', edgecolors = 'k', alpha=0.5, s=20*pourcentage, label=(str("   ")+str(pourcentage))+ ' %')
plt.legend(scatterpoints=1, frameon=False,labelspacing=3.5, loc='center left',title="Taux d'exportations en 2013");
# Prod + import + var stocks (baisse ou augmentation) = quantités disponibles pour l'exportation et l'utilisation intérieure.
# Prod + import - export + var stocks (baisse ou augmentation) = quantités disponibles pour l'utilisation intérieure.
title = str("Ratio = Exportation / (Exportation & Utilisation intérieure) de ") + produit + str(' en 2013')
plt.title(title,fontsize=20)

#affichage de qlq données significtaives (nom zone)
dv_vf_dispo_sample = dv_vf_dispo[dv_vf_dispo['Ratio_export_dispo'] > 0.66]
n=len(dv_vf_dispo_sample)
for i in range (n) :
    longi = dv_vf_dispo_sample.iloc[i,dv_vf_dispo_sample.columns.get_loc('Longitude')] 
    lati  = dv_vf_dispo_sample.iloc[i,dv_vf_dispo_sample.columns.get_loc('Latitude')] 
    zo    = dv_vf_dispo_sample.iloc[i,dv_vf_dispo_sample.columns.get_loc('Zone')] 
    plt.plot(longi,lati, 'ok', markersize=3)
    plt.text(longi,lati, zo, fontsize=10);
    
plt.text(-20,-60,'Coloré par exportation absolue', fontsize=10, color='k')    
plt.savefig('OUTPUTS_FORMATION/P3_image_ratio_exp_disp_manioc.jpg',dpi=300)
plt.show()

# Ratio Exportations / Quantité pour utilisation intérieure d'Huile de Palme en 2013

In [ ]:
#Soient les 4 produits les plus exportés identifiés en Q11 + quelqu'autres significatifs
list_produits =['Blé','Maïs','Lait - Excl Beurre','Soja','Manioc','Huile de Palme']
dv_vf_dispo = df_vf.loc[(df_vf['Produit']=='Huile de Palme') ,'Zone':'Longitude']
produit=dv_vf_dispo['Produit'].unique()
np.zeros((300, 250, 3), dtype = int)

# Extraction des data à visualiser
lat = dv_vf_dispo['Latitude'].values.astype(int)
lon = dv_vf_dispo['Longitude'].values
dispo_int   = dv_vf_dispo['Disponibilité intérieure'].values 
population  = dv_vf_dispo['Valeur_populationx1000'].values*1000                 #unité millions
production  = dv_vf_dispo['Production'].values                                  #milliers de tonne
exportation = dv_vf_dispo['Exportations - Quantité'].values                     #milliers de tonne
importation = dv_vf_dispo['Importations - Quantité'].values                     #milliers de tonne
varstock    = dv_vf_dispo['Variation de stock'].values                          #milliers de tonne
pourcentage = exportation/(exportation+dispo_int)

# création [Ratio] 
for i, row in dv_vf_dispo.iterrows():
    try: 
        dv_vf_dispo.loc[i,'Ratio_export_dispo'] = row['Exportations - Quantité']/(row['Exportations - Quantité']+row['Disponibilité intérieure'])
    except ZeroDivisionError:
        dv_vf_dispo.loc[i,'Ratio_export_dispo'] = 0
        
# préparation du background de la map
fig = plt.figure(figsize=(20, 15))
# m = Basemap(projection='cyl',llcrnrlat=-45,urcrnrlat=45,llcrnrlon=-15,urcrnrlon=60,resolution='c') # by zone
m = Basemap(projection='cyl', resolution='c', lon_0=0) # whole map
m.etopo(scale=0.5, alpha=0.5)
m.shadedrelief()
m.drawcoastlines(color='gray')
m.drawcountries(color='gray')
m.drawstates(color='gray')
       
# Echelle de couleur population par pays/capitale + taille pour sous-alimentation
m.scatter(lon,lat,latlon=True,c=np.log10(exportation),s=20*100*pourcentage,cmap='Reds',edgecolors = 'k',alpha=0.7)

# création colorbar + légende
# plt.colorbar(orientation="horizontal", label=r'$\log_{10}({\rm exportation})$')
bornm = np.log10(1)
bornp = np.log10(df_vf['Exportations - Quantité'].max())
plt.clim(bornm,bornp)

#affecter légende avec dummy points
for pourcentage in [33, 66, 100]: #(/!\ attention au facteur d'échelle imposé dans le "m.scatter")
    plt.scatter([], [],  c='k', edgecolors = 'k', alpha=0.5, s=20*pourcentage, label=(str("   ")+str(pourcentage))+ ' %')
plt.legend(scatterpoints=1, frameon=False,labelspacing=3.5, loc='center left',title="Taux d'exportations en 2013");
# Prod + import + var stocks (baisse ou augmentation) = quantités disponibles pour l'exportation et l'utilisation intérieure.
# Prod + import - export + var stocks (baisse ou augmentation) = quantités disponibles pour l'utilisation intérieure.
title = str("Ratio = Exportation / (Exportation & Utilisation intérieure) du ") + produit + str(' en 2013')
plt.title(title,fontsize=20)

#affichage de qlq données significtaives (nom zone)
dv_vf_dispo_sample = dv_vf_dispo[dv_vf_dispo['Ratio_export_dispo'] > 0.66]
n=len(dv_vf_dispo_sample)
for i in range (n) :
    longi = dv_vf_dispo_sample.iloc[i,dv_vf_dispo_sample.columns.get_loc('Longitude')] 
    lati  = dv_vf_dispo_sample.iloc[i,dv_vf_dispo_sample.columns.get_loc('Latitude')] 
    zo    = dv_vf_dispo_sample.iloc[i,dv_vf_dispo_sample.columns.get_loc('Zone')] 
    plt.plot(longi,lati, 'ok', markersize=3)
    plt.text(longi,lati, zo, fontsize=10);
    
plt.text(-20,-60,'Coloré par exportation absolue', fontsize=10, color='k')    
plt.savefig('OUTPUTS_FORMATION/P3_image_ratio_exp_disp_huile_palme.jpg',dpi=300)
plt.show()

# Ratio Exportations / Quantité pour utilisation intérieure de Soja en 2013

In [ ]:
#Soient les 4 produits les plus exportés identifiés en Q11 + quelqu'autres significatifs
list_produits =['Blé','Maïs','Lait - Excl Beurre','Soja','Manioc','Huile de Palme']
dv_vf_dispo = df_vf.loc[(df_vf['Produit']=='Soja') ,'Zone':'Longitude']
produit=dv_vf_dispo['Produit'].unique()
np.zeros((300, 250, 3), dtype = int)

# Extraction des data à visualiser
lat = dv_vf_dispo['Latitude'].values.astype(int)
lon = dv_vf_dispo['Longitude'].values
dispo_int   = dv_vf_dispo['Disponibilité intérieure'].values 
population  = dv_vf_dispo['Valeur_populationx1000'].values*1000                 #unité millions
production  = dv_vf_dispo['Production'].values                                  #milliers de tonne
exportation = dv_vf_dispo['Exportations - Quantité'].values                     #milliers de tonne
importation = dv_vf_dispo['Importations - Quantité'].values                     #milliers de tonne
varstock    = dv_vf_dispo['Variation de stock'].values                          #milliers de tonne
pourcentage = exportation/(exportation+dispo_int)

# création [Ratio] 
for i, row in dv_vf_dispo.iterrows():
    try: 
        dv_vf_dispo.loc[i,'Ratio_export_dispo'] = row['Exportations - Quantité']/(row['Exportations - Quantité']+row['Disponibilité intérieure'])
    except ZeroDivisionError:
        dv_vf_dispo.loc[i,'Ratio_export_dispo'] = 0
        
# préparation du background de la map
fig = plt.figure(figsize=(20, 15))
# m = Basemap(projection='cyl',llcrnrlat=-45,urcrnrlat=45,llcrnrlon=-15,urcrnrlon=60,resolution='c') # by zone
m = Basemap(projection='cyl', resolution='c', lon_0=0) # whole map
m.etopo(scale=0.5, alpha=0.5)
m.shadedrelief()
m.drawcoastlines(color='gray')
m.drawcountries(color='gray')
m.drawstates(color='gray')
       
# Echelle de couleur population par pays/capitale + taille pour sous-alimentation
m.scatter(lon,lat,latlon=True,c=np.log10(exportation),s=20*100*pourcentage,cmap='Reds',edgecolors = 'k',alpha=0.7)

# création colorbar + légende
# plt.colorbar(orientation="horizontal", label=r'$\log_{10}({\rm exportation})$')
bornm = np.log10(1)
bornp = np.log10(df_vf['Exportations - Quantité'].max())
plt.clim(bornm,bornp)

#affecter légende avec dummy points
for pourcentage in [33, 66, 100]: #(/!\ attention au facteur d'échelle imposé dans le "m.scatter")
    plt.scatter([], [],  c='k', edgecolors = 'k', alpha=0.5, s=20*pourcentage, label=(str("   ")+str(pourcentage))+ ' %')
plt.legend(scatterpoints=1, frameon=False,labelspacing=3.5, loc='center left',title="Taux d'exportations en 2013");
# Prod + import + var stocks (baisse ou augmentation) = quantités disponibles pour l'exportation et l'utilisation intérieure.
# Prod + import - export + var stocks (baisse ou augmentation) = quantités disponibles pour l'utilisation intérieure.
title = str("Ratio = Exportation / (Exportation & Utilisation intérieure) du ") + produit + str(' en 2013')
plt.title(title,fontsize=20)

#affichage de qlq données significtaives (nom zone)
dv_vf_dispo_sample = dv_vf_dispo[dv_vf_dispo['Ratio_export_dispo'] > 0.66]
n=len(dv_vf_dispo_sample)
for i in range (n) :
    longi = dv_vf_dispo_sample.iloc[i,dv_vf_dispo_sample.columns.get_loc('Longitude')] 
    lati  = dv_vf_dispo_sample.iloc[i,dv_vf_dispo_sample.columns.get_loc('Latitude')] 
    zo    = dv_vf_dispo_sample.iloc[i,dv_vf_dispo_sample.columns.get_loc('Zone')] 
    plt.plot(longi,lati, 'ok', markersize=3)
    plt.text(longi,lati, zo, fontsize=10);

plt.text(-20,-60,'Coloré par exportation absolue', fontsize=10, color='k')    
plt.savefig('OUTPUTS_FORMATION/P3_image_ratio_exp_disp_soja.jpg',dpi=300)
plt.show()

# Ratio Exportations / Quantité pour utilisation intérieure de Maïs en 2013

In [ ]:
#Soient les 4 produits les plus exportés identifiés en Q11 + quelqu'autres significatifs
list_produits =['Blé','Maïs','Lait - Excl Beurre','Soja','Manioc','Huile de Palme']
dv_vf_dispo = df_vf.loc[(df_vf['Produit']=='Maïs') ,'Zone':'Longitude']
produit=dv_vf_dispo['Produit'].unique()
np.zeros((300, 250, 3), dtype = int)

# Extraction des data à visualiser
lat = dv_vf_dispo['Latitude'].values.astype(int)
lon = dv_vf_dispo['Longitude'].values
dispo_int   = dv_vf_dispo['Disponibilité intérieure'].values 
population  = dv_vf_dispo['Valeur_populationx1000'].values*1000                 #unité millions
production  = dv_vf_dispo['Production'].values                                  #milliers de tonne
exportation = dv_vf_dispo['Exportations - Quantité'].values                     #milliers de tonne
importation = dv_vf_dispo['Importations - Quantité'].values                     #milliers de tonne
varstock    = dv_vf_dispo['Variation de stock'].values                          #milliers de tonne
pourcentage = exportation/(exportation+dispo_int)

# création [Ratio] 
for i, row in dv_vf_dispo.iterrows():
    try: 
        dv_vf_dispo.loc[i,'Ratio_export_dispo'] = row['Exportations - Quantité']/(row['Exportations - Quantité']+row['Disponibilité intérieure'])
    except ZeroDivisionError:
        dv_vf_dispo.loc[i,'Ratio_export_dispo'] = 0
        
# préparation du background de la map
fig = plt.figure(figsize=(20, 15))
# m = Basemap(projection='cyl',llcrnrlat=-45,urcrnrlat=45,llcrnrlon=-15,urcrnrlon=60,resolution='c') # by zone
m = Basemap(projection='cyl', resolution='c', lon_0=0) # whole map
m.etopo(scale=0.5, alpha=0.5)
m.shadedrelief()
m.drawcoastlines(color='gray')
m.drawcountries(color='gray')
m.drawstates(color='gray')
       
# Echelle de couleur indicateur 2 + taille indicateur 1 
m.scatter(lon,lat,latlon=True,c=np.log10(exportation),s=20*100*pourcentage,cmap='Reds',edgecolors = 'k',alpha=0.7)

# création colorbar + légende
# plt.colorbar(orientation="horizontal", label=r'$\log_{10}({\rm exportation})$')
bornm = np.log10(1)
bornp = np.log10(dv_vf_dispo['Exportations - Quantité'].max())
plt.clim(bornm,bornp)

#affecter légende avec dummy points
for pourcentage in [33, 66, 100]: #(/!\ attention au facteur d'échelle imposé dans le "m.scatter")
    plt.scatter([], [],  c='k', edgecolors = 'k', alpha=0.5, s=20*pourcentage, label=(str("   ")+str(pourcentage))+ ' %')
plt.legend(scatterpoints=1, frameon=False,labelspacing=3.5, loc='center left',title="Taux d'exportations en 2013");
# Prod + import + var stocks (baisse ou augmentation) = quantités disponibles pour l'exportation et l'utilisation intérieure.
# Prod + import - export + var stocks (baisse ou augmentation) = quantités disponibles pour l'utilisation intérieure.
title = str("Ratio = Exportation / (Exportation & Utilisation intérieure) du ") + produit + str(' en 2013')
plt.title(title,fontsize=20)

#affichage de qlq données significtaives (nom zone)
dv_vf_dispo_sample = dv_vf_dispo[dv_vf_dispo['Ratio_export_dispo'] > 0.66]
n=len(dv_vf_dispo_sample)
for i in range (n) :
    longi = dv_vf_dispo_sample.iloc[i,dv_vf_dispo_sample.columns.get_loc('Longitude')] 
    lati  = dv_vf_dispo_sample.iloc[i,dv_vf_dispo_sample.columns.get_loc('Latitude')] 
    zo    = dv_vf_dispo_sample.iloc[i,dv_vf_dispo_sample.columns.get_loc('Zone')] 
    plt.plot(longi,lati, 'ok', markersize=3)
    plt.text(longi,lati, zo, fontsize=10);

plt.text(-20,-60,'Coloré par exportation absolue', fontsize=10, color='k')    
plt.savefig('OUTPUTS_FORMATION/P3_image_ratio_exp_disp_maïs.jpg',dpi=300)
plt.show()

# Taux de Dépendance aux Importations de blé en 2013

In [ ]:
#Soient les 4 produits les plus exportés identifiés en Q11 + quelqu'autres significatifs
list_produits =['Blé','Maïs','Lait - Excl Beurre','Soja','Manioc','Huile de Palme']
dv_vf_dispo = df_vf.loc[(df_vf['Produit']=='Blé') ,'Zone':'Longitude']
produit=dv_vf_dispo['Produit'].unique()
np.zeros((300, 250, 3), dtype = int)

# dv_vf_dispo['Importations - Quantité'] = dv_vf_dispo['Importations - Quantité'].replace(0 ,1) #pour contourner log10(0)

# Création [] qui serviront de résultats & conditions ci-dessous
for i, row in dv_vf_dispo.iterrows():
    try: 
        dv_vf_dispo.loc[i,'Expor_over_Impor'] = row['Exportations - Quantité']/row['Importations - Quantité']
        dv_vf_dispo.loc[i,'TDI'] = row['Importations - Quantité']/(row['Importations - Quantité']+row['Production']-row['Exportations - Quantité'])
    except ZeroDivisionError:
        dv_vf_dispo.loc[i,'Expor_over_Impor'] = 0
        dv_vf_dispo.loc[i,'TDI'] = 0

# /!\ Zones à exclure car plus de la moitié jusqu'à la totalité importée est réexportée (pris en cpte si tx < 50%)
dv_vf_dispo = dv_vf_dispo[dv_vf_dispo['Expor_over_Impor'] < 0.001]

# Extraction des data à visualiser
lat = dv_vf_dispo['Latitude'].values.astype(int)
lon = dv_vf_dispo['Longitude'].values
dispo_int   = dv_vf_dispo['Disponibilité intérieure'].values 
population  = dv_vf_dispo['Valeur_populationx1000'].values*1000                 #unité millions
production  = dv_vf_dispo['Production'].values                                  #milliers de tonne
exportation = dv_vf_dispo['Exportations - Quantité'].values                     #milliers de tonne
importation = dv_vf_dispo['Importations - Quantité'].values                     #milliers de tonne
varstock    = dv_vf_dispo['Variation de stock'].values                          #milliers de tonne
tdi = importation/(importation+production-exportation)

# # création [Ratio] 
# for i, row in dv_vf_dispo.iterrows():
#     try:
#         dv_vf_dispo.loc[i,'TDI'] = row['Importations - Quantité']/(row['Importations - Quantité']+row['Production']-row['Exportations - Quantité'])
#     except ZeroDivisionError:
#         dv_vf_dispo.loc[i,'TDI'] = 0
        
# préparation du background de la map
fig = plt.figure(figsize=(20, 15))
# m = Basemap(projection='cyl',llcrnrlat=-45,urcrnrlat=45,llcrnrlon=-15,urcrnrlon=60,resolution='c') # by zone
m = Basemap(projection='cyl', resolution='c', lon_0=0) # whole map
m.etopo(scale=0.5, alpha=0.5)
m.shadedrelief()
m.drawcoastlines(color='gray')
m.drawcountries(color='gray')
m.drawstates(color='gray')
       
# Echelle de couleur population par pays/capitale + taille pour sous-alimentation
m.scatter(lon,lat,latlon=True,c=(tdi),s=5*100*tdi,cmap='Reds',edgecolors = 'k',alpha=0.7)

# création colorbar + légende
# plt.colorbar(orientation="horizontal", label=r'tdi')
bornm = (0)
bornp = (dv_vf_dispo['TDI'].max())
plt.clim(bornm,bornp)

#affecter légende avec dummy points
for tdi in [25, 50, 75, 100]: #(/!\ attention au facteur d'échelle imposé dans le "m.scatter")
    plt.scatter([], [],  c='k', edgecolors = 'k', alpha=0.5, s=5*tdi, label=(str("   ")+str(tdi))+ ' %')
plt.legend(scatterpoints=1, frameon=False,labelspacing=2, loc='lower left',title="TDI en 2013");
title = str("Taux de Dépendance aux Importations de ") + produit + str(' en 2013')
plt.title(title,fontsize=20)

#affichage de qlq données significtaives (nom zone)
dv_vf_dispo_sample = dv_vf_dispo[dv_vf_dispo['TDI'] > 1.45]
n=len(dv_vf_dispo_sample)
for i in range (n) :
    longi = dv_vf_dispo_sample.iloc[i,dv_vf_dispo_sample.columns.get_loc('Longitude')] 
    lati  = dv_vf_dispo_sample.iloc[i,dv_vf_dispo_sample.columns.get_loc('Latitude')] 
    zo    = dv_vf_dispo_sample.iloc[i,dv_vf_dispo_sample.columns.get_loc('Zone')] 
    plt.plot(longi,lati, 'ok', markersize=3)
    plt.text(longi,lati, zo, fontsize=10);
    
plt.text(-20,-60,'Coloré par TDI absolue', fontsize=10, color='k')
plt.text(-20,-65,'Seuls les Pays où les importations ne sont pas réexportées, sont affichés ici', fontsize=10, color='k')

plt.savefig('OUTPUTS_FORMATION/P3_image_TDI_blé.jpg',dpi=300)
plt.show()

# Taux de Dépendance aux Importations de Lait en 2013

In [ ]:
#Soient les 4 produits les plus exportés identifiés en Q11 + quelqu'autres significatifs
list_produits =['Blé','Maïs','Lait - Excl Beurre','Soja','Manioc','Huile de Palme']
dv_vf_dispo = df_vf.loc[(df_vf['Produit']=='Lait - Excl Beurre') ,'Zone':'Longitude']
produit=dv_vf_dispo['Produit'].unique()
np.zeros((300, 250, 3), dtype = int)

# dv_vf_dispo['Importations - Quantité'] = dv_vf_dispo['Importations - Quantité'].replace(0 ,1) #pour contourner log10(0)

# /!\ Zones à exclure car plus de la moitié jusqu'à la totalité importée est réexportée
for i, row in dv_vf_dispo.iterrows():
    try: 
        dv_vf_dispo.loc[i,'Expor_over_Impor'] = row['Exportations - Quantité']/row['Importations - Quantité']
        dv_vf_dispo.loc[i,'TDI'] = row['Importations - Quantité']/(row['Importations - Quantité']+row['Production']-row['Exportations - Quantité'])
    except ZeroDivisionError:
        dv_vf_dispo.loc[i,'Expor_over_Impor'] = 0
        dv_vf_dispo.loc[i,'TDI'] = 0

# /!\ Zones à exclure car plus de la moitié jusqu'à la totalité importée est réexportée (pris en cpte si tx < 50%)
dv_vf_dispo = dv_vf_dispo[dv_vf_dispo['Expor_over_Impor'] < 0.001]

# Extraction des data à visualiser
lat = dv_vf_dispo['Latitude'].values.astype(int)
lon = dv_vf_dispo['Longitude'].values
dispo_int   = dv_vf_dispo['Disponibilité intérieure'].values 
population  = dv_vf_dispo['Valeur_populationx1000'].values*1000                 #unité millions
production  = dv_vf_dispo['Production'].values                                  #milliers de tonne
exportation = dv_vf_dispo['Exportations - Quantité'].values                     #milliers de tonne
importation = dv_vf_dispo['Importations - Quantité'].values                     #milliers de tonne
varstock    = dv_vf_dispo['Variation de stock'].values                          #milliers de tonne
tdi = importation/(importation+production-exportation)

# # création [Ratio] 
# for i, row in dv_vf_dispo.iterrows():
#     try:
#         dv_vf_dispo.loc[i,'TDI'] = row['Importations - Quantité']/(row['Importations - Quantité']+row['Production']-row['Exportations - Quantité'])
#     except ZeroDivisionError:
#         dv_vf_dispo.loc[i,'TDI'] = 0
        
# préparation du background de la map
fig = plt.figure(figsize=(20, 15))
# m = Basemap(projection='cyl',llcrnrlat=-45,urcrnrlat=45,llcrnrlon=-15,urcrnrlon=60,resolution='c') # by zone
m = Basemap(projection='cyl', resolution='c', lon_0=0) # whole map
m.etopo(scale=0.5, alpha=0.5)
m.shadedrelief()
m.drawcoastlines(color='gray')
m.drawcountries(color='gray')
m.drawstates(color='gray')
       
# Echelle de couleur population par pays/capitale + taille pour sous-alimentation
m.scatter(lon,lat,latlon=True,c=(tdi),s=5*100*tdi,cmap='Reds',edgecolors = 'k',alpha=0.7)

# création colorbar + légende
# plt.colorbar(orientation="horizontal", label=r'tdi')
bornm = (0)
bornp = (dv_vf_dispo['TDI'].max())
plt.clim(bornm,bornp)

#affecter légende avec dummy points
for tdi in [25, 50, 75, 100]: #(/!\ attention au facteur d'échelle imposé dans le "m.scatter")
    plt.scatter([], [],  c='k', edgecolors = 'k', alpha=0.5, s=5*tdi, label=(str("   ")+str(tdi))+ ' %')
plt.legend(scatterpoints=1, frameon=False,labelspacing=2, loc='lower left',title="TDI en 2013");
title = str("Taux de Dépendance aux Importations de ") + produit + str(' en 2013')
plt.title(title,fontsize=20)

#affichage de qlq données significtaives (nom zone)
dv_vf_dispo_sample = dv_vf_dispo[dv_vf_dispo['TDI'] > 1.]
n=len(dv_vf_dispo_sample)
for i in range (n) :
    longi = dv_vf_dispo_sample.iloc[i,dv_vf_dispo_sample.columns.get_loc('Longitude')] 
    lati  = dv_vf_dispo_sample.iloc[i,dv_vf_dispo_sample.columns.get_loc('Latitude')] 
    zo    = dv_vf_dispo_sample.iloc[i,dv_vf_dispo_sample.columns.get_loc('Zone')] 
    plt.plot(longi,lati, 'ok', markersize=3)
    plt.text(longi,lati, zo, fontsize=10);
    
plt.text(-20,-60,'Coloré par TDI absolue', fontsize=10, color='k')
plt.text(-20,-65,'Seuls les Pays où les importations ne sont pas réexportées, sont affichés ici', fontsize=10, color='k')

plt.savefig('OUTPUTS_FORMATION/P3_image_TDI_lait.jpg',dpi=300)
plt.show()

# Taux de Dépendance aux Importations de Manioc en 2013

In [ ]:
#Soient les 4 produits les plus exportés identifiés en Q11 + quelqu'autres significatifs
list_produits =['Blé','Maïs','Lait - Excl Beurre','Soja','Manioc','Huile de Palme']
dv_vf_dispo = df_vf.loc[(df_vf['Produit']=='Manioc') ,'Zone':'Longitude']
produit=dv_vf_dispo['Produit'].unique()
np.zeros((300, 250, 3), dtype = int)

# dv_vf_dispo['Importations - Quantité'] = dv_vf_dispo['Importations - Quantité'].replace(0 ,1) #pour contourner log10(0)

# /!\ Zones à exclure car plus de la moitié jusqu'à la totalité importée est réexportée
for i, row in dv_vf_dispo.iterrows():
    try: 
        dv_vf_dispo.loc[i,'Expor_over_Impor'] = row['Exportations - Quantité']/row['Importations - Quantité']
        dv_vf_dispo.loc[i,'TDI'] = row['Importations - Quantité']/(row['Importations - Quantité']+row['Production']-row['Exportations - Quantité'])
    except ZeroDivisionError:
        dv_vf_dispo.loc[i,'Expor_over_Impor'] = 0
        dv_vf_dispo.loc[i,'TDI'] = 0

# /!\ Zones à exclure car plus de la moitié jusqu'à la totalité importée est réexportée (pris en cpte si tx < 50%)
dv_vf_dispo = dv_vf_dispo[dv_vf_dispo['Expor_over_Impor'] < 0.001]

# Extraction des data à visualiser
lat = dv_vf_dispo['Latitude'].values.astype(int)
lon = dv_vf_dispo['Longitude'].values
dispo_int   = dv_vf_dispo['Disponibilité intérieure'].values 
population  = dv_vf_dispo['Valeur_populationx1000'].values*1000                 #unité millions
production  = dv_vf_dispo['Production'].values                                  #milliers de tonne
exportation = dv_vf_dispo['Exportations - Quantité'].values                     #milliers de tonne
importation = dv_vf_dispo['Importations - Quantité'].values                     #milliers de tonne
varstock    = dv_vf_dispo['Variation de stock'].values                          #milliers de tonne
tdi = importation/(importation+production-exportation)


# # création [Ratio] 
# for i, row in dv_vf_dispo.iterrows():
#     try:
#         dv_vf_dispo.loc[i,'TDI'] = row['Importations - Quantité']/(row['Importations - Quantité']+row['Production']-row['Exportations - Quantité'])
#     except ZeroDivisionError:
#         dv_vf_dispo.loc[i,'TDI'] = 0
        
# préparation du background de la map
fig = plt.figure(figsize=(20, 15))
# m = Basemap(projection='cyl',llcrnrlat=-45,urcrnrlat=45,llcrnrlon=-15,urcrnrlon=60,resolution='c') # by zone
m = Basemap(projection='cyl', resolution='c', lon_0=0) # whole map
m.etopo(scale=0.5, alpha=0.5)
m.shadedrelief()
m.drawcoastlines(color='gray')
m.drawcountries(color='gray')
m.drawstates(color='gray')
       
# Echelle de couleur population par pays/capitale + taille pour sous-alimentation
m.scatter(lon,lat,latlon=True,c=(tdi),s=5*100*tdi,cmap='Reds',edgecolors = 'k',alpha=0.7)

# création colorbar + légende
# plt.colorbar(orientation="horizontal", label=r'tdi')
bornm = (0)
bornp = (dv_vf_dispo['TDI'].max())
plt.clim(bornm,bornp)

#affecter légende avec dummy points
for tdi in [25, 50, 75, 100]: #(/!\ attention au facteur d'échelle imposé dans le "m.scatter")
    plt.scatter([], [],  c='k', edgecolors = 'k', alpha=0.5, s=5*tdi, label=(str("   ")+str(tdi))+ ' %')
plt.legend(scatterpoints=1, frameon=False,labelspacing=2, loc='lower left',title="TDI en 2013");
title = str("Taux de Dépendance aux Importations de ") + produit + str(' en 2013')
plt.title(title,fontsize=20)

#affichage de qlq données significtaives (nom zone)
dv_vf_dispo_sample = dv_vf_dispo[dv_vf_dispo['TDI'] > 1.1]
n=len(dv_vf_dispo_sample)
for i in range (n) :
    longi = dv_vf_dispo_sample.iloc[i,dv_vf_dispo_sample.columns.get_loc('Longitude')] 
    lati  = dv_vf_dispo_sample.iloc[i,dv_vf_dispo_sample.columns.get_loc('Latitude')] 
    zo    = dv_vf_dispo_sample.iloc[i,dv_vf_dispo_sample.columns.get_loc('Zone')] 
    plt.plot(longi,lati, 'ok', markersize=3)
    plt.text(longi,lati, zo, fontsize=10);
    
plt.text(-20,-60,'Coloré par TDI absolue', fontsize=10, color='k')
plt.text(-20,-65,'Seuls les Pays où les importations ne sont pas réexportées, sont affichés ici', fontsize=10, color='k')

plt.savefig('OUTPUTS_FORMATION/P3_image_TDI_manioc.jpg',dpi=300)
plt.show()

# Taux de Dépendance aux Importations d'Huile de Palme en 2013

In [ ]:
#Soient les 4 produits les plus exportés identifiés en Q11 + quelqu'autres significatifs
list_produits =['Blé','Maïs','Lait - Excl Beurre','Soja','Manioc','Huile de Palme']
dv_vf_dispo = df_vf.loc[(df_vf['Produit']=='Huile de Palme') ,'Zone':'Longitude']
produit=dv_vf_dispo['Produit'].unique()
np.zeros((300, 250, 3), dtype = int)

# dv_vf_dispo['Importations - Quantité'] = dv_vf_dispo['Importations - Quantité'].replace(0 ,1) #pour contourner log10(0)

# /!\ Zones à exclure car plus de la moitié jusqu'à la totalité importée est réexportée
for i, row in dv_vf_dispo.iterrows():
    try: 
        dv_vf_dispo.loc[i,'Expor_over_Impor'] = row['Exportations - Quantité']/row['Importations - Quantité']
        dv_vf_dispo.loc[i,'TDI'] = row['Importations - Quantité']/(row['Importations - Quantité']+row['Production']-row['Exportations - Quantité'])
    except ZeroDivisionError:
        dv_vf_dispo.loc[i,'Expor_over_Impor'] = 0
        dv_vf_dispo.loc[i,'TDI'] = 0

# /!\ Zones à exclure car plus de la moitié jusqu'à la totalité importée est réexportée (pris en cpte si tx < 50%)
dv_vf_dispo = dv_vf_dispo[dv_vf_dispo['Expor_over_Impor'] < 0.001]

# Extraction des data à visualiser
lat = dv_vf_dispo['Latitude'].values.astype(int)
lon = dv_vf_dispo['Longitude'].values
dispo_int   = dv_vf_dispo['Disponibilité intérieure'].values 
population  = dv_vf_dispo['Valeur_populationx1000'].values*1000                 #unité millions
production  = dv_vf_dispo['Production'].values                                  #milliers de tonne
exportation = dv_vf_dispo['Exportations - Quantité'].values                     #milliers de tonne
importation = dv_vf_dispo['Importations - Quantité'].values                     #milliers de tonne
varstock    = dv_vf_dispo['Variation de stock'].values                          #milliers de tonne
tdi = importation/(importation+production-exportation)

# # création [Ratio] 
# for i, row in dv_vf_dispo.iterrows():
#     try:
#         dv_vf_dispo.loc[i,'TDI'] = row['Importations - Quantité']/(row['Importations - Quantité']+row['Production']-row['Exportations - Quantité'])
#     except ZeroDivisionError:
#         dv_vf_dispo.loc[i,'TDI'] = 0
        
# préparation du background de la map
fig = plt.figure(figsize=(20, 15))
# m = Basemap(projection='cyl',llcrnrlat=-45,urcrnrlat=45,llcrnrlon=-15,urcrnrlon=60,resolution='c') # by zone
m = Basemap(projection='cyl', resolution='c', lon_0=0) # whole map
m.etopo(scale=0.5, alpha=0.5)
m.shadedrelief()
m.drawcoastlines(color='gray')
m.drawcountries(color='gray')
m.drawstates(color='gray')
       
# Echelle de couleur population par pays/capitale + taille pour sous-alimentation
m.scatter(lon,lat,latlon=True,c=tdi,s=5*100*tdi,cmap='Reds',edgecolors = 'k',alpha=0.7)

# création colorbar + légende
# plt.colorbar(orientation="horizontal", label=r'tdi')
bornm = (0)
bornp = (dv_vf_dispo['TDI'].max())
plt.clim(bornm,bornp)

#affecter légende avec dummy points
for tdi in [25, 50, 75, 100]: #(/!\ attention au facteur d'échelle imposé dans le "m.scatter")
    plt.scatter([], [],  c='k', edgecolors = 'k', alpha=0.5, s=5*tdi, label=(str("   ")+str(tdi))+ ' %')
plt.legend(scatterpoints=1, frameon=False,labelspacing=2, loc='lower left',title="TDI en 2013");
title = str("Taux de Dépendance aux Importations de ") + produit + str(' en 2013')
plt.title(title,fontsize=20)

#affichage de qlq données significtaives (nom zone)
dv_vf_dispo_sample = dv_vf_dispo[dv_vf_dispo['TDI'] > 1.5]
n=len(dv_vf_dispo_sample)
for i in range (n) :
    longi = dv_vf_dispo_sample.iloc[i,dv_vf_dispo_sample.columns.get_loc('Longitude')] 
    lati  = dv_vf_dispo_sample.iloc[i,dv_vf_dispo_sample.columns.get_loc('Latitude')] 
    zo    = dv_vf_dispo_sample.iloc[i,dv_vf_dispo_sample.columns.get_loc('Zone')] 
    plt.plot(longi,lati, 'ok', markersize=3)
    plt.text(longi,lati, zo, fontsize=10);
    
plt.text(-20,-60,'Coloré par TDI absolue', fontsize=10, color='k')
plt.text(-20,-65,'Seuls les Pays où les importations ne sont pas réexportées, sont affichés ici', fontsize=10, color='k')

plt.savefig('DOUTPUTS_FORMATION/P3_image_TDI_huile_palme.jpg',dpi=300)
plt.show()

# Taux d'Autosuffisance en blé en 2013

In [ ]:
#Soient les 4 produits les plus exportés identifiés en Q11 + quelqu'autres significatifs
list_produits =['Blé','Maïs','Lait - Excl Beurre','Soja','Manioc','Huile de Palme']
dv_vf_dispo = df_vf.loc[(df_vf['Produit']=='Blé') ,'Zone':'Longitude']
produit=dv_vf_dispo['Produit'].unique()
np.zeros((300, 250, 3), dtype = int)

# dv_vf_dispo['Importations - Quantité'] = dv_vf_dispo['Importations - Quantité'].replace(0 ,1) #pour contourner log10(0)

# Création [] qui serviront de résultats & conditions ci-dessous
for i, row in dv_vf_dispo.iterrows():
    try: 
        dv_vf_dispo.loc[i,'Expor_over_Prod'] = row['Exportations - Quantité']/row['Production']
        dv_vf_dispo.loc[i,'TAS'] = row['Production']/(row['Importations - Quantité']+row['Production']-row['Exportations - Quantité'])
    except ZeroDivisionError:
        dv_vf_dispo.loc[i,'Expor_over_Prod'] = 0
        dv_vf_dispo.loc[i,'TAS'] = 0

# /!\ Zones à exclure car plus de la moitié jusqu'à la totalité produite est exportée (pris en cpte si tx < 50%)
dv_vf_dispo = dv_vf_dispo[dv_vf_dispo['Expor_over_Prod'] < 0.001]

# Extraction des data à visualiser
lat = dv_vf_dispo['Latitude'].values.astype(int)
lon = dv_vf_dispo['Longitude'].values
dispo_int   = dv_vf_dispo['Disponibilité intérieure'].values 
population  = dv_vf_dispo['Valeur_populationx1000'].values*1000                 #unité millions
production  = dv_vf_dispo['Production'].values                                  #milliers de tonne
exportation = dv_vf_dispo['Exportations - Quantité'].values                     #milliers de tonne
importation = dv_vf_dispo['Importations - Quantité'].values                     #milliers de tonne
varstock    = dv_vf_dispo['Variation de stock'].values                          #milliers de tonne
tas = production/(importation+production-exportation)

# # création [Ratio] 
# for i, row in dv_vf_dispo.iterrows():
#     try:
#         dv_vf_dispo.loc[i,'TDI'] = row['Importations - Quantité']/(row['Importations - Quantité']+row['Production']-row['Exportations - Quantité'])
#     except ZeroDivisionError:
#         dv_vf_dispo.loc[i,'TDI'] = 0
        
# préparation du background de la map
fig = plt.figure(figsize=(20, 15))
# m = Basemap(projection='cyl',llcrnrlat=-45,urcrnrlat=45,llcrnrlon=-15,urcrnrlon=60,resolution='c') # by zone
m = Basemap(projection='cyl', resolution='c', lon_0=0) # whole map
m.etopo(scale=0.5, alpha=0.5)
m.shadedrelief()
m.drawcoastlines(color='gray')
m.drawcountries(color='gray')
m.drawstates(color='gray')
       
# Echelle de couleur population par pays/capitale + taille pour sous-alimentation
m.scatter(lon,lat,latlon=True,c=(tas),s=5*100*tas,cmap='Reds',edgecolors = 'k',alpha=0.7)

# création colorbar + légende
# plt.colorbar(orientation="horizontal", label=r'tdi')
bornm = (0)
bornp = (dv_vf_dispo['TAS'].max())
plt.clim(bornm,bornp)

#affecter légende avec dummy points
for tas in [25, 50, 75, 100]: #(/!\ attention au facteur d'échelle imposé dans le "m.scatter")
    plt.scatter([], [],  c='k', edgecolors = 'k', alpha=0.5, s=5*tas, label=(str("   ")+str(tas))+ ' %')
plt.legend(scatterpoints=1, frameon=False,labelspacing=2, loc='lower left',title="TAS en 2013");
title = str("Taux d'Autosuffisance en ") + produit + str(' en 2013')
plt.title(title,fontsize=20)

#affichage de qlq données significtaives (nom zone)
dv_vf_dispo_sample = dv_vf_dispo[dv_vf_dispo['TAS'] > 1.45]
n=len(dv_vf_dispo_sample)
for i in range (n) :
    longi = dv_vf_dispo_sample.iloc[i,dv_vf_dispo_sample.columns.get_loc('Longitude')] 
    lati  = dv_vf_dispo_sample.iloc[i,dv_vf_dispo_sample.columns.get_loc('Latitude')] 
    zo    = dv_vf_dispo_sample.iloc[i,dv_vf_dispo_sample.columns.get_loc('Zone')] 
    plt.plot(longi,lati, 'ok', markersize=3)
    plt.text(longi,lati, zo, fontsize=10);
    
plt.text(-20,-60,'Coloré par TAS absolue', fontsize=10, color='k')
plt.text(-20,-65,"Seuls les Pays où la production n'est pas exportée, sont affichés ici", fontsize=10, color='k')

plt.savefig('OUTPUTS_FORMATION/P3_image_TAS_blé.jpg',dpi=300)
plt.show()

# Taux d'Autosuffisance en Lait en 2013

In [ ]:
#Soient les 4 produits les plus exportés identifiés en Q11 + quelqu'autres significatifs
list_produits =['Blé','Maïs','Lait - Excl Beurre','Soja','Manioc','Huile de Palme']
dv_vf_dispo = df_vf.loc[(df_vf['Produit']=='Lait - Excl Beurre') ,'Zone':'Longitude']
produit=dv_vf_dispo['Produit'].unique()
np.zeros((300, 250, 3), dtype = int)

# dv_vf_dispo['Importations - Quantité'] = dv_vf_dispo['Importations - Quantité'].replace(0 ,1) #pour contourner log10(0)

# Création [] qui serviront de résultats & conditions ci-dessous
for i, row in dv_vf_dispo.iterrows():
    try: 
        dv_vf_dispo.loc[i,'Expor_over_Prod'] = row['Exportations - Quantité']/row['Production']
        dv_vf_dispo.loc[i,'TAS'] = row['Production']/(row['Importations - Quantité']+row['Production']-row['Exportations - Quantité'])
    except ZeroDivisionError:
        dv_vf_dispo.loc[i,'Expor_over_Prod'] = 0
        dv_vf_dispo.loc[i,'TAS'] = 0

# /!\ Zones à exclure car plus de la moitié jusqu'à la totalité produite est exportée (pris en cpte si tx < 50%)
dv_vf_dispo = dv_vf_dispo[dv_vf_dispo['Expor_over_Prod'] < 0.001]

# Extraction des data à visualiser
lat = dv_vf_dispo['Latitude'].values.astype(int)
lon = dv_vf_dispo['Longitude'].values
dispo_int   = dv_vf_dispo['Disponibilité intérieure'].values 
population  = dv_vf_dispo['Valeur_populationx1000'].values*1000                 #unité millions
production  = dv_vf_dispo['Production'].values                                  #milliers de tonne
exportation = dv_vf_dispo['Exportations - Quantité'].values                     #milliers de tonne
importation = dv_vf_dispo['Importations - Quantité'].values                     #milliers de tonne
varstock    = dv_vf_dispo['Variation de stock'].values                          #milliers de tonne
tas = production/(importation+production-exportation)

# # création [Ratio] 
# for i, row in dv_vf_dispo.iterrows():
#     try:
#         dv_vf_dispo.loc[i,'TDI'] = row['Importations - Quantité']/(row['Importations - Quantité']+row['Production']-row['Exportations - Quantité'])
#     except ZeroDivisionError:
#         dv_vf_dispo.loc[i,'TDI'] = 0
        
# préparation du background de la map
fig = plt.figure(figsize=(20, 15))
# m = Basemap(projection='cyl',llcrnrlat=-45,urcrnrlat=45,llcrnrlon=-15,urcrnrlon=60,resolution='c') # by zone
m = Basemap(projection='cyl', resolution='c', lon_0=0) # whole map
m.etopo(scale=0.5, alpha=0.5)
m.shadedrelief()
m.drawcoastlines(color='gray')
m.drawcountries(color='gray')
m.drawstates(color='gray')
       
# Echelle de couleur population par pays/capitale + taille pour sous-alimentation
m.scatter(lon,lat,latlon=True,c=(tas),s=5*100*tas,cmap='Reds',edgecolors = 'k',alpha=0.7)

# création colorbar + légende
# plt.colorbar(orientation="horizontal", label=r'tdi')
bornm = (0)
bornp = (dv_vf_dispo['TAS'].max())
plt.clim(bornm,bornp)

#affecter légende avec dummy points
for tas in [25, 50, 75, 100]: #(/!\ attention au facteur d'échelle imposé dans le "m.scatter")
    plt.scatter([], [],  c='k', edgecolors = 'k', alpha=0.5, s=5*tas, label=(str("   ")+str(tas))+ ' %')
plt.legend(scatterpoints=1, frameon=False,labelspacing=2, loc='lower left',title="TAS en 2013");
title = str("Taux d'Autosuffisance en ") + produit + str(' en 2013')
plt.title(title,fontsize=20)

#affichage de qlq données significtaives (nom zone)
dv_vf_dispo_sample = dv_vf_dispo[dv_vf_dispo['TAS'] > 1.45]
n=len(dv_vf_dispo_sample)
for i in range (n) :
    longi = dv_vf_dispo_sample.iloc[i,dv_vf_dispo_sample.columns.get_loc('Longitude')] 
    lati  = dv_vf_dispo_sample.iloc[i,dv_vf_dispo_sample.columns.get_loc('Latitude')] 
    zo    = dv_vf_dispo_sample.iloc[i,dv_vf_dispo_sample.columns.get_loc('Zone')] 
    plt.plot(longi,lati, 'ok', markersize=3)
    plt.text(longi,lati, zo, fontsize=10);
    
plt.text(-20,-60,'Coloré par TAS absolue', fontsize=10, color='k')
# plt.text(-20,-65,'Les Pays dont plus de la moitié de la production est exportée ne sont pas affichés sur la map', fontsize=10, color='k')
plt.text(-20,-65,"Seuls les Pays où la production n'est pas exportée, sont affichés ici", fontsize=10, color='k')

plt.savefig('OUTPUTS_FORMATION/P3_image_TAS_lait.jpg',dpi=300)
plt.show()

# Taux d'Autosuffisance en Manioc en 2013

In [ ]:
#Soient les 4 produits les plus exportés identifiés en Q11 + quelqu'autres significatifs
list_produits =['Blé','Maïs','Lait - Excl Beurre','Soja','Manioc','Huile de Palme']
dv_vf_dispo = df_vf.loc[(df_vf['Produit']=='Manioc') ,'Zone':'Longitude']
produit=dv_vf_dispo['Produit'].unique()
np.zeros((300, 250, 3), dtype = int)

# dv_vf_dispo['Production'] = dv_vf_dispo['Production'].replace(0 ,0.1) #pour contourner log10(0)

# Création [] qui serviront de résultats & conditions ci-dessous
for i, row in dv_vf_dispo.iterrows():
    try: 
        dv_vf_dispo.loc[i,'Expor_over_Prod'] = row['Exportations - Quantité']/row['Production']
        dv_vf_dispo.loc[i,'TAS'] = row['Production']/(row['Importations - Quantité']+row['Production']-row['Exportations - Quantité'])
    except ZeroDivisionError:
        dv_vf_dispo.loc[i,'Expor_over_Prod'] = 0
        dv_vf_dispo.loc[i,'TAS'] = 0

# /!\ Zones à exclure car plus de la moitié jusqu'à la totalité produite est exportée (pris en cpte si tx < 50%)
dv_vf_dispo = dv_vf_dispo[dv_vf_dispo['Expor_over_Prod'] < 0.001]

# Extraction des data à visualiser
lat = dv_vf_dispo['Latitude'].values.astype(int)
lon = dv_vf_dispo['Longitude'].values
dispo_int   = dv_vf_dispo['Disponibilité intérieure'].values 
population  = dv_vf_dispo['Valeur_populationx1000'].values*1000                 #unité millions
production  = dv_vf_dispo['Production'].values                                  #milliers de tonne
exportation = dv_vf_dispo['Exportations - Quantité'].values                     #milliers de tonne
importation = dv_vf_dispo['Importations - Quantité'].values                     #milliers de tonne
varstock    = dv_vf_dispo['Variation de stock'].values                          #milliers de tonne
tas = production/(importation+production-exportation)

# # création [Ratio] 
# for i, row in dv_vf_dispo.iterrows():
#     try:
#         dv_vf_dispo.loc[i,'TDI'] = row['Importations - Quantité']/(row['Importations - Quantité']+row['Production']-row['Exportations - Quantité'])
#     except ZeroDivisionError:
#         dv_vf_dispo.loc[i,'TDI'] = 0
        
# préparation du background de la map
fig = plt.figure(figsize=(20, 15))
# m = Basemap(projection='cyl',llcrnrlat=-45,urcrnrlat=45,llcrnrlon=-15,urcrnrlon=60,resolution='c') # by zone
m = Basemap(projection='cyl', resolution='c', lon_0=0) # whole map
m.etopo(scale=0.5, alpha=0.5)
m.shadedrelief()
m.drawcoastlines(color='gray')
m.drawcountries(color='gray')
m.drawstates(color='gray')
       
# Echelle de couleur population par pays/capitale + taille pour sous-alimentation
m.scatter(lon,lat,latlon=True,c=(tas),s=5*100*tas,cmap='Reds',edgecolors = 'k',alpha=0.7)

# création colorbar + légende
# plt.colorbar(orientation="horizontal", label=r'tdi')
bornm = (0)
bornp = (dv_vf_dispo['TAS'].max())
plt.clim(bornm,bornp)

#affecter légende avec dummy points
for tas in [25, 50, 75, 100]: #(/!\ attention au facteur d'échelle imposé dans le "m.scatter")
    plt.scatter([], [],  c='k', edgecolors = 'k', alpha=0.5, s=5*tas, label=(str("   ")+str(tas))+ ' %')
plt.legend(scatterpoints=1, frameon=False,labelspacing=2, loc='lower left',title="TAS en 2013");
title = str("Taux d'Autosuffisance en ") + produit + str(' en 2013')
plt.title(title,fontsize=20)

#affichage de qlq données significtaives (nom zone)
dv_vf_dispo_sample = dv_vf_dispo[dv_vf_dispo['TAS'] > 1.01]
n=len(dv_vf_dispo_sample)
for i in range (n) :
    longi = dv_vf_dispo_sample.iloc[i,dv_vf_dispo_sample.columns.get_loc('Longitude')] 
    lati  = dv_vf_dispo_sample.iloc[i,dv_vf_dispo_sample.columns.get_loc('Latitude')] 
    zo    = dv_vf_dispo_sample.iloc[i,dv_vf_dispo_sample.columns.get_loc('Zone')] 
    plt.plot(longi,lati, 'ok', markersize=3)
    plt.text(longi,lati, zo, fontsize=10);
    
plt.text(-20,-60,'Coloré par TAS absolue', fontsize=10, color='k')
plt.text(-20,-65,"Seuls les Pays où la production n'est pas exportée, sont affichés ici", fontsize=10, color='k')

plt.savefig('OUTPUTS_FORMATION/P3_image_TAS_manioc.jpg',dpi=300)
plt.show()

# Taux d'Autosuffisance en Huile de Palme en 2013

In [ ]:
#Soient les 4 produits les plus exportés identifiés en Q11 + quelqu'autres significatifs
list_produits =['Blé','Maïs','Lait - Excl Beurre','Soja','Manioc','Huile de Palme']
dv_vf_dispo = df_vf.loc[(df_vf['Produit']=='Huile de Palme') ,'Zone':'Longitude']
produit=dv_vf_dispo['Produit'].unique()
np.zeros((300, 250, 3), dtype = int)

# dv_vf_dispo['Importations - Quantité'] = dv_vf_dispo['Importations - Quantité'].replace(0 ,1) #pour contourner log10(0)

# Création [] qui serviront de résultats & conditions ci-dessous
for i, row in dv_vf_dispo.iterrows():
    try: 
        dv_vf_dispo.loc[i,'Expor_over_Prod'] = row['Exportations - Quantité']/row['Production']
        dv_vf_dispo.loc[i,'TAS'] = row['Production']/(row['Importations - Quantité']+row['Production']-row['Exportations - Quantité'])
    except ZeroDivisionError:
        dv_vf_dispo.loc[i,'Expor_over_Prod'] = 0
        dv_vf_dispo.loc[i,'TAS'] = 0

# /!\ Zones à exclure car plus de la moitié jusqu'à la totalité produite est exportée (pris en cpte si tx < 50%)
dv_vf_dispo = dv_vf_dispo[dv_vf_dispo['Expor_over_Prod'] < 0.001] # RATIO EXPORT vs PROD

# Extraction des data à visualiser
lat = dv_vf_dispo['Latitude'].values.astype(int)
lon = dv_vf_dispo['Longitude'].values
dispo_int   = dv_vf_dispo['Disponibilité intérieure'].values 
population  = dv_vf_dispo['Valeur_populationx1000'].values*1000                 #unité millions
production  = dv_vf_dispo['Production'].values                                  #milliers de tonne
exportation = dv_vf_dispo['Exportations - Quantité'].values                     #milliers de tonne
importation = dv_vf_dispo['Importations - Quantité'].values                     #milliers de tonne
varstock    = dv_vf_dispo['Variation de stock'].values                          #milliers de tonne
tas = production/(importation+production-exportation)

# # création [Ratio] 
# for i, row in dv_vf_dispo.iterrows():
#     try:
#         dv_vf_dispo.loc[i,'TDI'] = row['Importations - Quantité']/(row['Importations - Quantité']+row['Production']-row['Exportations - Quantité'])
#     except ZeroDivisionError:
#         dv_vf_dispo.loc[i,'TDI'] = 0
        
# préparation du background de la map
fig = plt.figure(figsize=(20, 15))
# m = Basemap(projection='cyl',llcrnrlat=-45,urcrnrlat=45,llcrnrlon=-15,urcrnrlon=60,resolution='c') # by zone
m = Basemap(projection='cyl', resolution='c', lon_0=0) # whole map
m.etopo(scale=0.5, alpha=0.5)
m.shadedrelief()
m.drawcoastlines(color='gray')
m.drawcountries(color='gray')
m.drawstates(color='gray')
       
# Echelle de couleur population par pays/capitale + taille pour sous-alimentation
m.scatter(lon,lat,latlon=True,c=(tas),s=10*100*tas,cmap='Reds',edgecolors = 'k',alpha=0.7)

# création colorbar + légende
# plt.colorbar(orientation="horizontal", label=r'tdi')
bornm = (0)
bornp = (dv_vf_dispo['TAS'].max())
plt.clim(bornm,bornp)

#affecter légende avec dummy points
for tas in [25, 50, 75, 100]: #(/!\ attention au facteur d'échelle imposé dans le "m.scatter")
    plt.scatter([], [],  c='k', edgecolors = 'k', alpha=0.5, s=10*tas, label=(str("   ")+str(tas))+ ' %')
plt.legend(scatterpoints=1, frameon=False,labelspacing=2, loc='lower left',title="TAS en 2013");
title = str("Taux d'Autosuffisance en ") + produit + str(' en 2013')
plt.title(title,fontsize=20)

#affichage de qlq données significtaives (nom zone)
dv_vf_dispo_sample = dv_vf_dispo[dv_vf_dispo['TAS'] > .99]
n=len(dv_vf_dispo_sample)
for i in range (n) :
    longi = dv_vf_dispo_sample.iloc[i,dv_vf_dispo_sample.columns.get_loc('Longitude')] 
    lati  = dv_vf_dispo_sample.iloc[i,dv_vf_dispo_sample.columns.get_loc('Latitude')] 
    zo    = dv_vf_dispo_sample.iloc[i,dv_vf_dispo_sample.columns.get_loc('Zone')] 
    plt.plot(longi,lati, 'ok', markersize=3)
    plt.text(longi,lati, zo, fontsize=10);
    
plt.text(-20,-60,'Coloré par TAS absolue', fontsize=10, color='k')
plt.text(-20,-65,"Seuls les Pays où la production n'est pas exportée, sont affichés ici", fontsize=10, color='k')

plt.savefig('OUTPUTS_FORMATION/P3_image_TAS_huile_palme.jpg',dpi=300)
plt.show()

In [ ]:
# #identification des exportations significatives (>10 milliers de tonnes) 
# dv_vf_export_sample = dv_vf_export[dv_vf_export['Exportations - Quantité']>10] 
# n=len(dv_vf_export_sample) for i in range (n) : 
#     longi = dv_vf_export_sample.iloc[i,dv_vf_export_sample.columns.get_loc('Longitude')] 
#     lati = dv_vf_export_sample.iloc[i,dv_vf_export_sample.columns.get_loc('Latitude')] 
#     zo = dv_vf_export_sample.iloc[i,dv_vf_export_sample.columns.get_loc('Zone')] 
#     plt.plot(longi,lati, 'ok', markersize=3) 
#     plt.text(longi,lati, zo, fontsize=10);

In [ ]:
# #Soient les 4 produits les plus exportés identifiés en Q11
# list_produits =['Blé','Maïs','Lait - Excl Beurre','Soja']
# dv_vf_prodagr = df_vf.loc[(df_vf['Produit']=='Blé') ,'Zone':'Longitude']
# produit=dv_vf_prodagr['Produit'].unique()
# np.zeros((300, 250, 3), dtype = int)

# # Extraction des data à visualiser
# lat = dv_vf_prodagr['Latitude'].values.astype(int)
# lon = dv_vf_prodagr['Longitude'].values
# #conversion numérique
# dv_vf_prodagr['Aliments pour animaux'] = pd.to_numeric(dv_vf_prodagr['Aliments pour animaux'])
# dv_vf_prodagr['Nourriture'] = pd.to_numeric(dv_vf_prodagr['Nourriture'])
# dv_vf_prodagr['Pertes'] = pd.to_numeric(dv_vf_prodagr['Pertes'])

# alim_animx = dv_vf_prodagr['Aliments pour animaux'].values                    #milliers de tonne
# nourriture = dv_vf_prodagr['Nourriture'].values                                #milliers de tonne
# Pertes = dv_vf_prodagr['Pertes'].values                                         #milliers de tonne
# population = dv_vf_prodagr['Valeur_populationx1000'].values*1000                 #unité millions
# exportation = dv_vf_prodagr['Exportations - Quantité'].values                     #milliers de tonne

# # création [Production_Totale] contenant la somme des 3 contributions
# for i, row in dv_vf_prodagr.iterrows():
#     dv_vf_prodagr.loc[i,'Prod_agr_tot'] = row['Aliments pour animaux']+row['Nourriture']+row['Pertes']
# Sum_prod = dv_vf_prodagr['Prod_agr_tot'].values  

# # préparation du background de la map
# fig = plt.figure(figsize=(15, 15))
# m = Basemap(projection='cyl', resolution='c', lon_0=0) # whole map
# m.etopo(scale=0.5, alpha=0.5)
# m.shadedrelief()
# m.drawcoastlines(color='gray')
# m.drawcountries(color='gray')
# m.drawstates(color='gray')

# norm = colors.Normalize(Sum_prod.min(), Sum_prod.max())
# m.scatter(lon,lat,latlon=True,c=np.log10(Sum_prod), norm=norm,s=Sum_prod/10,cmap='Reds',edgecolors = 'k',alpha=0.7)

# # création colorbar + légende
# # plt.colorbar(orientation="horizontal", label=r'exportation')
# bornm = np.log10(1)
# bornp = np.log10(dv_vf_prodagr['Prod_agr_tot'].max())
# plt.clim(bornm,bornp)

# #affecter légende avec dummy points
# for Sum_prod in [2500, 5000, 10000]: #(/!\ attention au facteur d'échelle imposé dans le "m.scatter")
#         plt.scatter([], [],  c='k', edgecolors = 'k', alpha=0.5, s=Sum_prod, label=(str("      ")+str(Sum_prod))) #marker oversizing
# plt.legend(scatterpoints=1, frameon=False,labelspacing=5.5, loc='center left',title='Production agricole (x 10000 tonnes)');
# title = str('Production agricole mondiale de ') + produit + str(' en 2013')
# plt.title(title,fontsize=20)

# #affichage de qlq données significtaives (nom zone)
# dv_vf_prodagr_sample = dv_vf_prodagr[dv_vf_prodagr['Prod_agr_tot']>10000]
# n=len(dv_vf_prodagr_sample)
# for i in range (n) :
#     longi = dv_vf_prodagr_sample.iloc[i,dv_vf_prodagr_sample.columns.get_loc('Longitude')] 
#     lati  = dv_vf_prodagr_sample.iloc[i,dv_vf_prodagr_sample.columns.get_loc('Latitude')] 
#     zo    = dv_vf_prodagr_sample.iloc[i,dv_vf_prodagr_sample.columns.get_loc('Zone')] 
#     plt.plot(longi,lati, 'ok', markersize=3)
#     plt.text(longi,lati, zo, fontsize=10);

# plt.savefig('OUTPUTS_FORMATION/P3_image_production_blé.jpg',dpi=300)
# plt.show()